In [1]:
import carla
import argparse
import time
import signal
import sys
import traceback
import os
import subprocess
import re
import socket
import atexit
import threading
from datetime import datetime
from xml.etree import ElementTree as ET
import py_trees


# os.environ['CUDA_HOME'] = '/usr/local/cuda-12.2'
os.environ['IS_BENCH2DRIVE'] = 'true'
os.environ['REPETITION'] = '1'
# os.environ['DATAGEN'] = '1'


In [2]:
import subprocess, re

def kill_carla_processes_on_gpu(gpu_index: int = 0):
    """Kill CARLA processes on specific GPU"""
    try:
        raw = subprocess.check_output("nvidia-smi", shell=True).decode()
        pattern = rf'\|\s+{gpu_index}\s+\S+\s+\S+\s+(\d+)\s+C\+G\s+.*CarlaUE4'
        pids = re.findall(pattern, raw)
        if not pids:
            print(f"No CarlaUE4 processes found on GPU {gpu_index}")
            return
        for pid in pids:
            print(f"Killing CarlaUE4 process with PID {pid} on GPU {gpu_index}")
            subprocess.run(["kill", "-9", pid], check=True)
    except Exception as e:
        print(f"Error killing CARLA processes: {e}")

def kill_python_processes_on_gpu(gpu_index: int = 0):
    """Kill Python processes on specific GPU"""
    try:
        raw = subprocess.check_output("nvidia-smi", shell=True).decode()
        pattern = rf'\|\s+{gpu_index}\s+\S+\s+\S+\s+(\d+)\s+C\s+.*bin/python'
        pids = re.findall(pattern, raw)
        if not pids:
            print(f"No python processes found on GPU {gpu_index}")
            return
        for pid in pids:
            print(f"Killing python process with PID {pid} on GPU {gpu_index}")
            subprocess.run(["kill", "-9", pid], check=True)
    except Exception as e:
        print(f"Error killing Python processes: {e}")

def cleanup_gpu_processes():
    """Clean up GPU processes"""
    for gpu_id in [0, 1]:
        kill_carla_processes_on_gpu(gpu_id)
        kill_python_processes_on_gpu(gpu_id)

cleanup_gpu_processes()

Killing CarlaUE4 process with PID 359980 on GPU 0
No python processes found on GPU 0
No CarlaUE4 processes found on GPU 1
No python processes found on GPU 1


In [3]:
# Setup paths
leaderboard_path = '/mnt3/Documents/AD_Framework/Bench2Drive/leaderboard/leaderboard'
if leaderboard_path not in sys.path:
    sys.path.insert(0, leaderboard_path)

# Check environment variables
print("CARLA_ROOT: ", os.environ.get('CARLA_ROOT'))
print("WORK_DIR: ", os.environ.get('WORK_DIR'))
print("SCENARIO_RUNNER_ROOT: ", os.environ.get('SCENARIO_RUNNER_ROOT'))
print("LD_LIBRARY_PATH: ", os.environ.get('LD_LIBRARY_PATH'))
print("LEADERBOARD_ROOT: ", os.environ.get('LEADERBOARD_ROOT'))
print("CUDA_HOME: ", os.environ.get('CUDA_HOME'))
print("PYTHONPATH: ", os.environ.get('PYTHONPATH'))

CARLA_ROOT:  /mnt3/Documents/AD_Framework/carla0915
WORK_DIR:  /mnt3/Documents/AD_Framework/Bench2Drive
SCENARIO_RUNNER_ROOT:  /mnt3/Documents/AD_Framework/Bench2Drive/scenario_runner
LD_LIBRARY_PATH:  /usr/local/cuda-12.1/lib64:
LEADERBOARD_ROOT:  /mnt3/Documents/AD_Framework/Bench2Drive/leaderboard
CUDA_HOME:  /usr/local/cuda-12.1
PYTHONPATH:  /mnt3/Documents/AD_Framework/carla0915/PythonAPI/carla/:/mnt3/Documents/AD_Framework/Bench2Drive/scenario_runner:/mnt3/Documents/AD_Framework/Bench2Drive/leaderboard::/mnt3/Documents/AD_Framework/Bench2Drive:/mnt3/Documents/AD_Framework/carla0915/PythonAPI:/mnt3/Documents/AD_Framework/carla0915/PythonAPI/carla:/mnt3/Documents/AD_Framework/carla0915/PythonAPI/carla/:/mnt3/Documents/AD_Framework/Bench2Drive/scenario_runner:/mnt3/Documents/AD_Framework/Bench2Drive/leaderboard::/mnt3/Documents/AD_Framework/Bench2Drive:/mnt3/Documents/AD_Framework/carla0915/PythonAPI:/mnt3/Documents/AD_Framework/carla0915/PythonAPI/carla


In [4]:
port = 20019

model_name = 'my_model_mlp'
statistics_manager_path = f'/mnt3/Documents/AD_Framework/results/{model_name}'

# makedir if do not exist
os.makedirs(statistics_manager_path, exist_ok=True)

args = argparse.Namespace()
args.WORK_DIR = os.environ.get('WORK_DIR')
args.checkpoint = f'{statistics_manager_path}/stat_manager_results.json'
args.debug_checkpoint = f'{statistics_manager_path}/live_results.txt'
args.host = '127.0.0.3'
args.port = port
args.timeout = 6000
args.frame_rate = 20.0 # fixed
args.gpu_rank = 0
args.traffic_manager_port = port + 3
# '/po3/korawat/Documents/AD_Framework/Bench2Drive/leaderboard/leaderboard/autoagents/dummy_agent.py'
# args.TEAM_AGENT = f"{args.WORK_DIR}/leaderboard/team_code/data_agent2.py" # auto_pilot.py
args.TEAM_AGENT = f'{leaderboard_path}/autoagents/dummy_agent4.py'
## auto_pilot.py
args.agent = args.TEAM_AGENT
print("args.agent", args.agent)
args.agent_config = ''
args.debug = 1
args.routes = f"/mnt3/Documents/AD_Framework/Bench2Drive/leaderboard/data/bench2drive220.xml"
args.repetitions = 1
args.track = 'SENSORS' # Changed from 'MAP' to 'SENSORS' to allow camera sensors
args.record = ''
args.routes_subset = 0
args.resume = 0
args.traffic_manager_seed = port - 3333 + 1

# Enable camera sensors by setting required environment variables
os.environ['SAVE_PATH'] = statistics_manager_path
os.environ['IS_BENCH2DRIVE'] = 'true'
os.environ['TMP_VISU'] = '0'  # Enable camera sensors for visualization

print(f"SAVE_PATH set to: {statistics_manager_path}")
print(f"TMP_VISU set to: {os.environ.get('TMP_VISU')}")
print(f"Track set to: {args.track}")

args.agent /mnt3/Documents/AD_Framework/Bench2Drive/leaderboard/leaderboard/autoagents/dummy_agent4.py
SAVE_PATH set to: /mnt3/Documents/AD_Framework/results/my_model_mlp
TMP_VISU set to: 0
Track set to: SENSORS


In [5]:
from srunner.scenariomanager.carla_data_provider import *
from srunner.scenariomanager.timer import GameTime
from srunner.scenariomanager.watchdog import Watchdog

from leaderboard.scenarios.scenario_manager import ScenarioManager
from leaderboard.scenarios.route_scenario import RouteScenario
from leaderboard.envs.sensor_interface import SensorConfigurationInvalid
from leaderboard.autoagents.agent_wrapper import AgentError, validate_sensor_configuration, TickRuntimeError
from leaderboard.utils.statistics_manager import StatisticsManager, FAILURE_MESSAGES
from leaderboard.utils.route_indexer import RouteIndexer

# print("args: ", args)
statistics_manager = StatisticsManager(args.checkpoint , args.debug_checkpoint)
# statistics_manager.write_statistics() ## run one time

# leaderboard_evaluator = LeaderboardEvaluator(args, statistics_manager)

LeaderboardEvaluator_path = '/po3/korawat/Documents/AD_Framework/Bench2Drive/leaderboard/leaderboard/leaderboard_evaluator.py'
if LeaderboardEvaluator_path not in sys.path:
    sys.path.insert(0, LeaderboardEvaluator_path)

from leaderboard.leaderboard_evaluator import LeaderboardEvaluator


In [6]:
import socket
import subprocess
import atexit


def find_free_port(starting_port):
    """Find a free port starting from the given port"""
    port = starting_port
    while True:
        try:
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
                s.bind(("localhost", port))
                return port
        except OSError:
            port += 1

def _setup_simulation(self, args):
    """Setup simulation with CARLA server"""
    os.system("pkill -f CarlaUE4")
    time.sleep(5)

    self.carla_path = os.environ["CARLA_ROOT"]
    args.port = find_free_port(args.port)
    cmd1 = f"{os.path.join(self.carla_path, 'CarlaUE4.sh')} -RenderOffScreen -nosound -quality-level=Epic -carla-rpc-port={args.port} -carla-streaming-port=0 -graphicsadapter=0"
    self.server = subprocess.Popen(cmd1, shell=True, preexec_fn=os.setsid)
    atexit.register(os.killpg, self.server.pid, signal.SIGKILL)

    # Wait for server to be ready
    server_ready = False
    for _ in range(30):
        try:
            with socket.create_connection(('localhost', args.port), timeout=1):
                server_ready = True
                break
        except (ConnectionRefusedError, socket.timeout):
            time.sleep(5)
    if not server_ready:
        raise RuntimeError("Carla server failed to start")
        
    attempts = 0
    num_max_restarts = 20
    print("Loading world!!!")
    while attempts < num_max_restarts:
        try:
            client = carla.Client(args.host, args.port)
            if args.timeout:
                client_timeout = args.timeout
            client.set_timeout(client_timeout)

            settings = carla.WorldSettings(
                synchronous_mode = True,
                fixed_delta_seconds = 1.0 / self.frame_rate,
                deterministic_ragdolls = True,
                spectator_as_ego = False
            )
            client.get_world().apply_settings(settings)
            print(f"load_world success , attempts={attempts}", flush=True)
            break
        except Exception as e:
            print(f"load_world failed , attempts={attempts}", flush=True)
            print(e, flush=True)
            attempts += 1
            time.sleep(5)
    
    attempts = 0
    num_max_restarts = 40
    while attempts < num_max_restarts:
        try:
            args.traffic_manager_port = find_free_port(args.traffic_manager_port)
            traffic_manager = client.get_trafficmanager(args.traffic_manager_port)
            traffic_manager.set_synchronous_mode(True)
            traffic_manager.set_hybrid_physics_mode(True)
            print(f"traffic_manager init success, try_time={attempts}", flush=True)
            break
        except Exception as e:
            print(f"traffic_manager init fail, try_time={attempts}", flush=True)
            print(e, flush=True)
            attempts += 1
            time.sleep(5)

    return client, client_timeout, traffic_manager


def get_weather_id(weather_conditions):
    from xml.etree import ElementTree as ET
    WORK_DIR = os.environ.get("WORK_DIR", "")
    if WORK_DIR != "":
        WORK_DIR += "/"
    tree = ET.parse(WORK_DIR + 'leaderboard/data/weather.xml')
    root = tree.getroot()
    def conditions_match(weather, conditions):
        for (key, value) in weather:
            if key == 'route_percentage' : continue
            if str(getattr(conditions, key))!= value:
                return False
        return True
    for case in root.findall('case'):
        weather = case[0].items()
        if conditions_match(weather, weather_conditions):
            return case.items()[0][1]
    return None

sensors_to_icons = {
    'sensor.camera.rgb':        'carla_camera',
    'sensor.lidar.ray_cast':    'carla_lidar',
    'sensor.other.radar':       'carla_radar',
    'sensor.other.gnss':        'carla_gnss',
    'sensor.other.imu':         'carla_imu',
    'sensor.opendrive_map':     'carla_opendrive_map',
    'sensor.speedometer':       'carla_speedometer'
}

#### the following are the functions that equal to _load_and_run_scenario in leaderboard_evaluator.py

def my_load_scenario(leaderboard_evaluator_self, args, config):
    """Load and run the scenario given by config"""
    crash_message = ""
    entry_status = "Started"
    save_name = ""

    print("\n\033[1m========= Preparing {} (repetition {}) =========\033[0m".format(config.name, config.repetition_index), flush=True)

    # Prepare the statistics of the route
    route_name = f"{config.name}_rep{config.repetition_index}"
    scenario_name = config.scenario_configs[0].name
    town_name = str(config.town)
    weather_id = get_weather_id(config.weather[0][1])
    currentDateAndTime = datetime.now()
    currentTime = currentDateAndTime.strftime("%m_%d_%H_%M_%S")
    save_name = f"{route_name}_{town_name}_{scenario_name}_{weather_id}_{currentTime}"
    leaderboard_evaluator_self.statistics_manager.create_route_data(route_name, scenario_name, weather_id, save_name, town_name, config.index)

    print("\033[1m> Loading the world\033[0m", flush=True)

    # Load the world and the scenario
    try:
        leaderboard_evaluator_self._load_and_wait_for_world(args, config.town)
        from leaderboard.scenarios.route_scenario import RouteScenario
        leaderboard_evaluator_self.route_scenario = RouteScenario(world=leaderboard_evaluator_self.world, config=config, debug_mode=args.debug)
        leaderboard_evaluator_self.statistics_manager.set_scenario(leaderboard_evaluator_self.route_scenario)

    except Exception:
        # The scenario is wrong -> set the ejecution to crashed and stop
        print("\n\033[91mThe scenario could not be loaded:", flush=True)
        print(f"\n{traceback.format_exc()}\033[0m", flush=True)

        from leaderboard.utils.statistics_manager import FAILURE_MESSAGES
        entry_status, crash_message = FAILURE_MESSAGES["Simulation"]
        leaderboard_evaluator_self._register_statistics(config.index, entry_status, crash_message)
        leaderboard_evaluator_self._cleanup()
        return True, save_name, entry_status, crash_message

    return False, save_name, entry_status, crash_message

def my_load_agent(leaderboard_evaluator_self, args, config, save_name, entry_status, crash_message):
    """Set up the user's agent and configure sensors"""
    print("\033[1m> Setting up the agent\033[0m", flush=True)

    # Set up the user's agent, and the timer to avoid freezing the simulation
    try:
        from srunner.scenariomanager.watchdog import Watchdog
        from leaderboard.envs.sensor_interface import SensorConfigurationInvalid
        from leaderboard.autoagents.agent_wrapper import validate_sensor_configuration
        from leaderboard.utils.statistics_manager import FAILURE_MESSAGES
        
        leaderboard_evaluator_self._agent_watchdog = Watchdog(args.timeout)
        leaderboard_evaluator_self._agent_watchdog.start()
        agent_class_name = getattr(leaderboard_evaluator_self.module_agent, 'get_entry_point')()
        agent_class_obj = getattr(leaderboard_evaluator_self.module_agent, agent_class_name)

        # Start the ROS1 bridge server only for ROS1 based agents.
        if getattr(agent_class_obj, 'get_ros_version')() == 1 and leaderboard_evaluator_self._ros1_server is None:
            from leaderboard.autoagents.ros1_agent import ROS1Server
            leaderboard_evaluator_self._ros1_server = ROS1Server()
            leaderboard_evaluator_self._ros1_server.start()

        leaderboard_evaluator_self.agent_instance = agent_class_obj(args.host, args.port, args.debug)
        leaderboard_evaluator_self.agent_instance.set_global_plan(leaderboard_evaluator_self.route_scenario.gps_route, leaderboard_evaluator_self.route_scenario.route)
        args.agent_config = args.agent_config + '+' + save_name
        leaderboard_evaluator_self.agent_instance.setup(args.agent_config)

        # Check and store the sensors
        if not leaderboard_evaluator_self.sensors:
            leaderboard_evaluator_self.sensors = leaderboard_evaluator_self.agent_instance.sensors()
            track = leaderboard_evaluator_self.agent_instance.track

            validate_sensor_configuration(leaderboard_evaluator_self.sensors, track, args.track)

            leaderboard_evaluator_self.sensor_icons = [sensors_to_icons[sensor['type']] for sensor in leaderboard_evaluator_self.sensors]
            leaderboard_evaluator_self.statistics_manager.save_sensors(leaderboard_evaluator_self.sensor_icons)
            leaderboard_evaluator_self.statistics_manager.write_statistics()

            leaderboard_evaluator_self.sensors_initialized = True

        leaderboard_evaluator_self._agent_watchdog.stop()
        leaderboard_evaluator_self._agent_watchdog = None

    except SensorConfigurationInvalid as e:
        # The sensors are invalid -> set the ejecution to rejected and stop
        print("\n\033[91mThe sensor's configuration used is invalid:", flush=True)
        print(f"{e}\033[0m\n", flush=True)

        entry_status, crash_message = FAILURE_MESSAGES["Sensors"]
        leaderboard_evaluator_self._register_statistics(config.index, entry_status, crash_message)
        leaderboard_evaluator_self._cleanup()
        return True, entry_status, crash_message

    except Exception as e:
        # The agent setup has failed -> start the next route
        print("\n\033[91mCould not set up the required agent:", flush=True)
        print(f"\n{traceback.format_exc()}\033[0m", flush=True)
        print(f"{e}\033[0m\n", flush=True)

        entry_status, crash_message = FAILURE_MESSAGES["Agent_init"]
        leaderboard_evaluator_self._register_statistics(config.index, entry_status, crash_message)
        leaderboard_evaluator_self._cleanup()
        return True, entry_status, crash_message

    return False, entry_status, crash_message

def my_load_route_scenario(leaderboard_evaluator_self, args, config, entry_status, crash_message):
    """Load the route scenario and prepare for execution"""
    from leaderboard.utils.statistics_manager import FAILURE_MESSAGES
    
    print("\033[1m> Running the route\033[0m", flush=True)

    # Run the scenario
    try:
        # Load scenario and run it
        if args.record:
            leaderboard_evaluator_self.client.start_recorder("{}/{}_rep{}.log".format(args.record, config.name, config.repetition_index))
        leaderboard_evaluator_self.manager.load_scenario(leaderboard_evaluator_self.route_scenario, leaderboard_evaluator_self.agent_instance, config.index, config.repetition_index)
    except Exception:
        print("\n\033[91mFailed to load the scenario, the statistics might be empty:", flush=True)
        print("\n\033[91mLoading the route, the agent has crashed:", flush=True)
        entry_status, crash_message = FAILURE_MESSAGES["Agent_runtime"]

    return False, entry_status, crash_message

def my_run_scenario_setup(leaderboard_evaluator_self):
    """Trigger the start of the scenario and wait for it to finish/fail"""
    from srunner.scenariomanager.timer import GameTime
    from srunner.scenariomanager.watchdog import Watchdog
    
    leaderboard_evaluator_self.manager.tick_count = 0
    leaderboard_evaluator_self.manager.start_system_time = time.time()
    leaderboard_evaluator_self.manager.start_game_time = GameTime.get_time()

    # Detects if the simulation is down
    leaderboard_evaluator_self.manager._watchdog = Watchdog(leaderboard_evaluator_self.manager._timeout)
    leaderboard_evaluator_self.manager._watchdog.start()

    # Stop the agent from freezing the simulation
    leaderboard_evaluator_self.manager._agent_watchdog = Watchdog(leaderboard_evaluator_self.manager._timeout)
    leaderboard_evaluator_self.manager._agent_watchdog.start()

    leaderboard_evaluator_self.manager._running = True

    # Thread for build_scenarios
    leaderboard_evaluator_self.manager._scenario_thread = threading.Thread(target=leaderboard_evaluator_self.manager.build_scenarios_loop, args=(leaderboard_evaluator_self.manager._debug_mode > 0, ))
    leaderboard_evaluator_self.manager._scenario_thread.start()

def my_run_scenario_step(leaderboard_evaluator_self, entry_status, crash_message, n_steps=1):
    """Run n_steps of the scenario simulation"""
    from leaderboard.autoagents.agent_wrapper import AgentError, TickRuntimeError
    from leaderboard.utils.statistics_manager import FAILURE_MESSAGES
    
    try:
        if leaderboard_evaluator_self.manager._running:
            print("In my_run_scenario_step (Check if it is stuck or not)")
            for _ in range(n_steps):
                leaderboard_evaluator_self.manager._tick_scenario()

    except AgentError:
        # The agent has failed -> stop the route
        print("\n\033[91mStopping the route, the agent has crashed:", flush=True)
        print(f"\n{traceback.format_exc()}\033[0m")

        entry_status, crash_message = FAILURE_MESSAGES["Agent_runtime"]

        return True, entry_status, crash_message

    except KeyboardInterrupt:
        return True, entry_status, crash_message
    
    except TickRuntimeError:
        print("TickRuntimeError")
        entry_status, crash_message = "Started", "TickRuntime"
        return True, entry_status, crash_message
    
    except Exception:
        print("\n\033[91mError during the simulation:", flush=True)
        print(f"\n{traceback.format_exc()}\033[0m", flush=True)

        entry_status, crash_message = FAILURE_MESSAGES["Simulation"]
        return True, entry_status, crash_message
    
    return False, entry_status, crash_message

def my_stop_scenario(leaderboard_evaluator_self, args, config, entry_status, crash_message):
    """Stop the scenario and register statistics"""
    from leaderboard.utils.statistics_manager import FAILURE_MESSAGES
    
    # Stop the scenario
    try:
        print("\033[1m> Stopping the route\033[0m", flush=True)
        leaderboard_evaluator_self.manager.stop_scenario()
        leaderboard_evaluator_self._register_statistics(config.index, entry_status, crash_message)

        if args.record:
            leaderboard_evaluator_self.client.stop_recorder()

        leaderboard_evaluator_self._cleanup()

    except Exception:
        print("\n\033[91mFailed to stop the scenario, the statistics might be empty:", flush=True)
        print(f"\n{traceback.format_exc()}\033[0m", flush=True)

        _, crash_message = FAILURE_MESSAGES["Simulation"]

    return False, entry_status, crash_message

# replace _setup_simulation method of LeaderboardEvaluator with this new one 
LeaderboardEvaluator._setup_simulation = _setup_simulation


In [7]:
### Main code object

leaderboard_evaluator = LeaderboardEvaluator(args, statistics_manager)

4.26.2-0+++UE4+Release-4.26 522 0
Disabling core dumps.
Loading world!!!
load_world success , attempts=0
traffic_manager init success, try_time=0


In [8]:
print("args.routes", args.routes)
print("args.repetitions", args.repetitions)
print("args.routes_subset", args.routes_subset)

args.routes /mnt3/Documents/AD_Framework/Bench2Drive/leaderboard/data/bench2drive220.xml
args.repetitions 1
args.routes_subset 0


In [9]:
route_indexer = RouteIndexer(args.routes, args.repetitions, args.routes_subset)
if args.resume:
    resume = route_indexer.validate_and_resume(args.checkpoint)
else:
    resume = False

if resume:
    leaderboard_evaluator.statistics_manager.add_file_records(args.checkpoint)
else:
    leaderboard_evaluator.statistics_manager.clear_records()
leaderboard_evaluator.statistics_manager.save_progress(route_indexer.index, route_indexer.total)
leaderboard_evaluator.statistics_manager.write_statistics()

In [10]:
crashed_flag = False
t1 = time.time()
route_count = 0

In [11]:
### in the loop of running

want_list = ['1790'] #['25845']

for i in range(250):
    config = route_indexer.get_next_config()
    if config is None:
        print("No more routes to process")
        

    route_id = config.name.split("_")[1]
    print("route_id: ", route_id)
    if route_id in want_list and len(want_list) > 0:
        print("Got the route_id: ", route_id)
        break
    elif len(want_list) == 0:
        route_id = route_id
        print("Got the route_id: ", route_id)
        break
    
print(f"\n=== Processing Route {route_count + 1}/{route_indexer.total} ===")
print(f"Route: {config.name}, Town: {config.town}", "Route_index: ", route_indexer.index)

### want route_id = 2509

route_id:  1711
route_id:  1773
route_id:  1790
Got the route_id:  1790

=== Processing Route 1/220 ===
Route: RouteScenario_1790, Town: Town12 Route_index:  3


In [12]:
# Step 1: Load scenario

crash_flag, save_name, entry_status, crash_message = my_load_scenario(leaderboard_evaluator, args, config)
print("crash_flag: ", crash_flag)
print("save_name: ", save_name)
print("entry_status: ", entry_status)
print("crash_message: ", crash_message)

save_name = save_name + '_' + model_name


========= Preparing RouteScenario_1790 (repetition 0) =========
> Loading the world
crash_flag:  False
save_name:  RouteScenario_1790_rep0_Town12_HazardAtSideLane_1_8_09_03_02_33_04
entry_status:  Started
crash_message:  


In [13]:
# Step 2: Load agent
if not crash_flag:
    crash_flag, entry_status, crash_message = my_load_agent(leaderboard_evaluator, args, config, save_name, entry_status, crash_message)

print("crash_flag: ", crash_flag)
print("entry_status: ", entry_status)
print("crash_message: ", crash_message)

> Setting up the agent
DummyAgent2 initialized - will save 10 images to /mnt3/Documents/AD_Framework/town_test_images
crash_flag:  False
entry_status:  Started
crash_message:  


In [14]:
# Step 3: Load route scenario
if not crash_flag:
    crash_flag, entry_status, crash_message = my_load_route_scenario(leaderboard_evaluator, args, config, entry_status, crash_message)

print("crash_flag: ", crash_flag)
print("entry_status: ", entry_status)
print("crash_message: ", crash_message)

> Running the route
crash_flag:  False
entry_status:  Started
crash_message:  


In [15]:
# Step 4: Setup scenario
if not crash_flag:
    my_run_scenario_setup(leaderboard_evaluator)

# Add image saving capability to the agent
import numpy as np
from datetime import datetime

# Monkey-patch the agent to capture sensor data
original_run_step = leaderboard_evaluator.agent_instance.run_step

def enhanced_run_step(input_data, timestamp):
    """Enhanced run_step that captures sensor data for image saving"""
    # Store the input data for later access
    leaderboard_evaluator.agent_instance.last_input_data = input_data
    leaderboard_evaluator.agent_instance.step = getattr(leaderboard_evaluator.agent_instance, 'step', 0) + 1
    
    # Call original run_step
    return original_run_step(input_data, timestamp)

# Replace the run_step method
leaderboard_evaluator.agent_instance.run_step = enhanced_run_step
leaderboard_evaluator.agent_instance.step = 0

print("Agent enhanced with image capture capability")

Agent enhanced with image capture capability


In [16]:
# print("leaderboard_evaluator.manager._running", leaderboard_evaluator.manager._running)

# leaderboard_evaluator.manager._running = True
# if leaderboard_evaluator.manager._running:
#     print("In my_run_scenario_step (Check if it is stuck or not)")
#     for _ in range(1):
#         leaderboard_evaluator.manager._tick_scenario()

# print("leaderboard_evaluator.manager._running after", leaderboard_evaluator.manager._running)


In [17]:
# Step 5: Run scenario loop step by step with automatic image saving
import cv2
import numpy as np
from pathlib import Path
from datetime import datetime

leaderboard_evaluator.manager._running = True

# Create save directory with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
save_dir = Path(f"/mnt3/Documents/AD_Framework/bench2drive-gymnasium/bench2drive_microservices/notebooks/debug_snapshots/debug_images_{timestamp}")
save_dir.mkdir(parents=True, exist_ok=True)
print(f"Images will be saved to: {save_dir}")

def save_debug_images(agent_instance, save_dir, step_count):
    """Save all sensor images for debugging with velocity overlay"""
    if not hasattr(agent_instance, 'last_input_data'):
        return

    input_data = agent_instance.last_input_data
    # leaderboard_evaluator.agent_instance.last_input_data['speed'][1]['speed']

    # Create step-specific subdirectory
    step_dir = save_dir / f"step_{step_count:04d}"
    step_dir.mkdir(exist_ok=True)

    # Extract velocity data from sensors
    ego_speed = None
    ego_velocity = None

    # Try to get speed from SPEED sensor
    if 'speed' in input_data:
        speed_data = input_data['speed']
        if isinstance(speed_data, tuple) and len(speed_data) >= 2:
            if isinstance(speed_data[1], dict) and 'speed' in speed_data[1]:
                ego_speed = speed_data[1]['speed']  # m/s
            elif isinstance(speed_data[1], (int, float)):
                ego_speed = float(speed_data[1])

    # Try to get velocity from IMU sensor
    if 'IMU' in input_data:
        imu_data = input_data['IMU']
        if isinstance(imu_data, tuple) and len(imu_data) >= 2:
            if isinstance(imu_data[1], dict):
                if 'accelerometer' in imu_data[1]:
                    # IMU provides acceleration, not velocity directly
                    pass

    # Try to get GPS data for position tracking
    gps_data = None
    if 'GPS' in input_data:
        gps = input_data['GPS']
        if isinstance(gps, tuple) and len(gps) >= 2:
            if isinstance(gps[1], dict):
                gps_data = gps[1]

    # Save RGB cameras with velocity overlay
    for camera_id in ['Center', 'Left', 'Right']:
        if camera_id in input_data:
            rgb_image = input_data[camera_id][1][:, :, :3]  # Remove alpha channel if present
            frame_num = input_data[camera_id][0]

            # Convert from RGB to BGR for OpenCV
            bgr_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR)

            # Add velocity overlay on Center camera
            if camera_id == 'Center':
                # Create overlay text
                overlay_text = []
                overlay_text.append(f"Step: {step_count:04d}")

                # Add ego vehicle speed
                if ego_speed is not None:
                    # Convert m/s to km/h for better readability
                    speed_kmh = ego_speed * 3.6
                    overlay_text.append(f"Ego Speed: {speed_kmh:.1f} km/h ({ego_speed:.1f} m/s)")
                else:
                    overlay_text.append("Ego Speed: N/A")

                # Add GPS info if available
                if gps_data:
                    lat = gps_data.get('lat', 0)
                    lon = gps_data.get('lon', 0)
                    alt = gps_data.get('alt', 0)
                    overlay_text.append(f"GPS: ({lat:.6f}, {lon:.6f}, {alt:.1f}m)")

                # Draw overlay on image
                font = cv2.FONT_HERSHEY_SIMPLEX
                font_scale = 0.7
                thickness = 2
                y_offset = 30

                for i, text in enumerate(overlay_text):
                    # Add black background for text visibility
                    (text_width, text_height), baseline = cv2.getTextSize(text, font, font_scale, thickness)
                    cv2.rectangle(bgr_image,
                                (10, y_offset + i*35 - text_height - 5),
                                (10 + text_width + 10, y_offset + i*35 +5),
                                (0, 0, 0), -1)

                    # Draw white text
                    cv2.putText(bgr_image, text,
                            (15, y_offset + i*35),
                            font, font_scale, (255, 255, 255), thickness)

            # Save image
            image_path = step_dir / f"{camera_id}_frame_{frame_num:06d}.jpg"
            cv2.imwrite(str(image_path), bgr_image)
            # print(f"  Saved: {camera_id} -> {image_path.name}")

    # Save sensor data info to text file including velocity
    info_path = step_dir / "sensor_info.txt"
    with open(info_path, 'w') as f:
        f.write(f"Step: {step_count}\n")
        f.write(f"Timestamp: {datetime.now()}\n")
        if ego_speed is not None:
            f.write(f"Ego Speed: {ego_speed:.2f} m/s ({ego_speed*3.6:.1f} km/h)\n")
        if gps_data:
            f.write(f"GPS: lat={gps_data.get('lat', 0):.6f}, lon={gps_data.get('lon', 0):.6f}, alt={gps_data.get('alt', 0):.1f}\n")
        f.write("\nSensor Data:\n")
        for key, val in input_data.items():
            if hasattr(val[1], 'shape'):
                f.write(f"  {key}: frame={val[0]:06d}, shape={val[1].shape}\n")
            else:
                f.write(f"  {key}: frame={val[0]:06d}\n")

    return step_dir


# Run simulation steps with image saving
run_step_n = 50  # run 5 steps = 0.25 second
total_steps_run = 0

if not crash_flag:
    print(f"\n=== Running {run_step_n} simulation steps with image capture ===")
    
    for step_i in range(run_step_n):
        # Run one step
        crash_flag_step, entry_status, crash_message = my_run_scenario_step(
            leaderboard_evaluator, entry_status, crash_message, n_steps=1
        )
        
        if crash_flag_step:
            crash_flag = True
            print(f"Crash detected at step {step_i}")
            break
        
        # Save images after each step
        if hasattr(leaderboard_evaluator.agent_instance, 'last_input_data'):
            print(f"\nStep {total_steps_run + step_i}:")
            step_dir = save_debug_images(
                leaderboard_evaluator.agent_instance, 
                save_dir, 
                total_steps_run + step_i
            )
        else:
            print(f"Step {total_steps_run + step_i}: No sensor data captured")
    
    total_steps_run += step_i + 1
    
print(f"\n=== Completed {total_steps_run} steps ===")
print(f"crash_flag: {crash_flag}")
print(f"entry_status: {entry_status}")
print(f"crash_message: {crash_message}")
print(f"Images saved to: {save_dir}")

# Display summary
if save_dir.exists():
    step_dirs = sorted(save_dir.glob("step_*"))
    print(f"\nSaved {len(step_dirs)} steps of data")
    for step_dir in step_dirs[:3]:  # Show first 3 steps
        images = list(step_dir.glob("*.jpg"))
        print(f"  {step_dir.name}: {len(images)} images")

Images will be saved to: /mnt3/Documents/AD_Framework/bench2drive-gymnasium/bench2drive_microservices/notebooks/debug_snapshots/debug_images_20250903_023356

=== Running 50 simulation steps with image capture ===
In my_run_scenario_step (Check if it is stuck or not)
=== [Agent] -- Wallclock = 2025-09-03 02:33:56.630 -- System time = 0.000 -- Game time = 0.050 -- Ratio = 0.000x
Testing Test_0 - Route_0
Frame 1/10 - Test_0
  Saving Center camera: Test_0_Route_0_Center_frame_001.png
  Saving Left camera: Test_0_Route_0_Left_frame_001.png
  Saving Right camera: Test_0_Route_0_Right_frame_001.png

Step 0:
In my_run_scenario_step (Check if it is stuck or not)
=== [Agent] -- Wallclock = 2025-09-03 02:33:57.436 -- System time = 0.806 -- Game time = 0.100 -- Ratio = 0.124x
Frame 2/10 - Test_0

Step 1:
In my_run_scenario_step (Check if it is stuck or not)
=== [Agent] -- Wallclock = 2025-09-03 02:33:58.153 -- System time = 1.523 -- Game time = 0.150 -- Ratio = 0.098x
Frame 3/10 - Test_0

Step 2:


In [18]:
import carla

def get_transform_info(carla_vehicle_object):
    transform = carla_vehicle_object.get_transform()
    print("transform.location.x", transform.location.x)
    print("transform.location.y", transform.location.y)
    print("transform.location.z", transform.location.z)

    print("transform.rotation.yaw", transform.rotation.yaw)
    print("transform.rotation.pitch", transform.rotation.pitch)
    print("transform.rotation.roll", transform.rotation.roll)
    vel = carla_vehicle_object.get_velocity()
    print("vel.x", vel.x, "vel.y", vel.y, "vel.z", vel.z)
    return transform

def set_carla_loc_rot(carla_vehicle_object, x, y, z, yaw=None, pitch=None, roll=None):
    loc = carla.Location(x=x, y=y, z=z)

    if yaw is not None or pitch is not None or roll is not None:
        cur_rot = carla_vehicle_object.get_transform().rotation
        rot = carla.Rotation(
            pitch=pitch if pitch is not None else cur_rot.pitch,
            yaw=yaw if yaw is not None else cur_rot.yaw,
            roll=roll if roll is not None else cur_rot.roll
        )
        carla_vehicle_object.set_transform(carla.Transform(loc, rot))
    else:
        carla_vehicle_object.set_location(loc)
    ## cannot check as it is require to start the step before resetting the position

get_transform_info(leaderboard_evaluator.manager.scenario.list_scenarios[0].other_actors[0])
print("----")
get_transform_info(leaderboard_evaluator.manager.scenario.list_scenarios[0].other_actors[1])
print("----")
get_transform_info(leaderboard_evaluator.manager.ego_vehicles[0])

transform.location.x 537.2979125976562
transform.location.y 3910.021484375
transform.location.z 371.21435546875
transform.rotation.yaw 179.72799682617188
transform.rotation.pitch -0.45734262466430664
transform.rotation.roll 0.001633104751817882
vel.x 2.413727315797587e-06 vel.y 0.0003448301868047565 vel.z 0.0020283746998757124
----
transform.location.x 528.2965698242188
transform.location.y 3910.070556640625
transform.location.z 371.2020568847656
transform.rotation.yaw 179.72560119628906
transform.rotation.pitch -0.21338193118572235
transform.rotation.roll 0.019010258838534355
vel.x -2.4318695068359375e-05 vel.y -0.00032338968594558537 vel.z 0.009335293434560299
----
transform.location.x 580.465087890625
transform.location.y 3910.699951171875
transform.location.z 371.2985534667969
transform.rotation.yaw 179.76072692871094
transform.rotation.pitch 0.7089735865592957
transform.rotation.roll 0.001819672528654337
vel.x -12.2544527053833 vel.y 0.05126480013132095 vel.z -0.008819589391350746

In [19]:
import json
import csv     

world_obj = leaderboard_evaluator.manager.scenario.world

def record_vehicles(world_obj, save_path=None, save_json=False):
    actors = world_obj.get_actors()
    vehicles = actors.filter("vehicle.*")

    records = []
    for v in vehicles:
        tf = v.get_transform()
        loc, rot = tf.location, tf.rotation
        vel = v.get_velocity()
        ang = v.get_angular_velocity()
        acc = v.get_acceleration()
        ctrl = v.get_control()

        record = {
            "id": v.id,
            "type_id": v.type_id,
            "x": loc.x,
            "y": loc.y,
            "z": loc.z,
            "pitch": rot.pitch,
            "yaw": rot.yaw,
            "roll": rot.roll,
            "vx": vel.x, "vy": vel.y, "vz": vel.z,
            "wx": ang.x, "wy": ang.y, "wz": ang.z,
            "ax": acc.x, "ay": acc.y, "az": acc.z,
            "throttle": ctrl.throttle,
            "steer": ctrl.steer,
            "brake": ctrl.brake,
            "hand_brake": ctrl.hand_brake,
            "reverse": ctrl.reverse,
            "gear": ctrl.gear,
        }
        records.append(record)

    if save_path:
        if save_json:
            import json
            with open(save_path, "w") as f:
                json.dump(records, f, indent=4)
            print(f"Saved {len(records)} vehicle records to {save_path} (JSON)")
        else:
            import csv
            with open(save_path, mode="w", newline="") as f:
                writer = csv.DictWriter(f, fieldnames=records[0].keys())
                writer.writeheader()
                writer.writerows(records)
            print(f"Saved {len(records)} vehicle records to {save_path} (CSV)")

    return records

def watchdog_save_state(watchdog_object):
    record_dict = {
        "timeout": watchdog_object._timeout,
        "interval": watchdog_object._interval,
        "failed": watchdog_object._failed,
        "stopped": watchdog_object._watchdog_stopped,
    }
    return record_dict

### to save state 
vehicles_state = record_vehicles(world_obj)
print("vehicles_state", vehicles_state)
print("--------------------------------")

watchdog_state = watchdog_save_state(leaderboard_evaluator.manager._watchdog)
print("watchdog_state", watchdog_state)
print("--------------------------------")
    

vehicles_state [{'id': 3771, 'type_id': 'vehicle.lincoln.mkz_2020', 'x': 529.6852416992188, 'y': 3920.765625, 'z': 371.2006530761719, 'pitch': 0.43278124928474426, 'yaw': 0.11760231107473373, 'roll': 0.016277821734547615, 'vx': 6.651047706604004, 'vy': 0.005099917761981487, 'vz': 0.025384750217199326, 'wx': -0.04707903042435646, 'wy': -4.131840229034424, 'wz': -0.38689517974853516, 'ax': 12.258825302124023, 'ay': -0.0012694299221038818, 'az': 0.03517858684062958, 'throttle': 0.8500000238418579, 'steer': -0.002491297898814082, 'brake': 0.0, 'hand_brake': False, 'reverse': False, 'gear': 2}, {'id': 3770, 'type_id': 'vehicle.mini.cooper_s_2021', 'x': 529.888671875, 'y': 3917.51708984375, 'z': 371.2105712890625, 'pitch': 0.18733158707618713, 'yaw': 0.0877789556980133, 'roll': -9.155272709904239e-05, 'vx': 6.486534118652344, 'vy': 0.002112228889018297, 'vz': 0.02569270133972168, 'wx': -0.00952806044369936, 'wy': 0.2599191665649414, 'wz': -0.8037283420562744, 'ax': 1.7831707000732422, 'ay': 

In [20]:
actors = world_obj.get_actors()
vehicles = actors.filter("vehicle.*")
for actor in vehicles:
    print("-----------")
    print("actor.id", actor.id)
    print("yaw:", actor.get_transform().rotation.yaw, "pitch:", actor.get_transform().rotation.pitch, "roll:", actor.get_transform().rotation.roll)
    print("speed",actor.get_velocity().x, actor.get_velocity().y, actor.get_velocity().z)

-----------
actor.id 3771
yaw: 0.11760231107473373 pitch: 0.43278124928474426 roll: 0.016277821734547615
speed 6.651047706604004 0.005099917761981487 0.025384750217199326
-----------
actor.id 3770
yaw: 0.0877789556980133 pitch: 0.18733158707618713 roll: -9.155272709904239e-05
speed 6.486534118652344 0.002112228889018297 0.02569270133972168
-----------
actor.id 3769
yaw: -179.79873657226562 pitch: 0.07573313266038895 roll: 0.002902447013184428
speed -6.340973854064941 -0.008854418061673641 0.006159028969705105
-----------
actor.id 3768
yaw: -179.91253662109375 pitch: 0.21467965841293335 roll: 0.005242487881332636
speed -8.929043769836426 -0.002869710326194763 0.0055673313327133656
-----------
actor.id 3767
yaw: 179.72750854492188 pitch: -0.29925790429115295 roll: 0.0
speed 0.0 0.0 0.0
-----------
actor.id 3766
yaw: 179.72750854492188 pitch: -0.3584551513195038 roll: 0.0
speed 0.0 0.0 0.0
-----------
actor.id 3765
yaw: -179.77146911621094 pitch: 0.07515256851911545 roll: 0.00232063909061

In [21]:
# actors = world_obj.get_actors()
# vehicles = actors.filter("vehicle.*")

In [22]:
# for v in vehicles:
#     print("v", v)

In [23]:

#### second 50s

# Create save directory with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
save_dir = Path(f"/mnt3/Documents/AD_Framework/bench2drive-gymnasium/bench2drive_microservices/notebooks/debug_snapshots/debug_images_{timestamp}")
save_dir.mkdir(parents=True, exist_ok=True)
print(f"Images will be saved to: {save_dir}")

run_step_n = 50  # run 5 steps = 0.25 second
total_steps_run = 0

if not crash_flag:
    print(f"\n=== Running {run_step_n} simulation steps with image capture ===")
    
    for step_i in range(run_step_n):
        # Run one step
        crash_flag_step, entry_status, crash_message = my_run_scenario_step(
            leaderboard_evaluator, entry_status, crash_message, n_steps=1
        )
        
        if crash_flag_step:
            crash_flag = True
            print(f"Crash detected at step {step_i}")
            break
        
        # Save images after each step
        if hasattr(leaderboard_evaluator.agent_instance, 'last_input_data'):
            print(f"\nStep {total_steps_run + step_i}:")
            step_dir = save_debug_images(
                leaderboard_evaluator.agent_instance, 
                save_dir, 
                total_steps_run + step_i
            )
        else:
            print(f"Step {total_steps_run + step_i}: No sensor data captured")
    
    total_steps_run += step_i + 1
    
print(f"\n=== Completed {total_steps_run} steps ===")
print(f"crash_flag: {crash_flag}")
print(f"entry_status: {entry_status}")
print(f"crash_message: {crash_message}")
print(f"Images saved to: {save_dir}")

Images will be saved to: /mnt3/Documents/AD_Framework/bench2drive-gymnasium/bench2drive_microservices/notebooks/debug_snapshots/debug_images_20250903_023411

=== Running 50 simulation steps with image capture ===
In my_run_scenario_step (Check if it is stuck or not)
=== [Agent] -- Wallclock = 2025-09-03 02:34:11.199 -- System time = 14.569 -- Game time = 2.550 -- Ratio = 0.175x
Frame 51/10 - Test_0

Step 0:
In my_run_scenario_step (Check if it is stuck or not)
=== [Agent] -- Wallclock = 2025-09-03 02:34:11.399 -- System time = 14.769 -- Game time = 2.600 -- Ratio = 0.176x
Frame 52/10 - Test_0

Step 1:
In my_run_scenario_step (Check if it is stuck or not)
=== [Agent] -- Wallclock = 2025-09-03 02:34:11.595 -- System time = 14.966 -- Game time = 2.650 -- Ratio = 0.177x
Frame 53/10 - Test_0

Step 2:
In my_run_scenario_step (Check if it is stuck or not)
=== [Agent] -- Wallclock = 2025-09-03 02:34:11.790 -- System time = 15.160 -- Game time = 2.700 -- Ratio = 0.178x
Frame 54/10 - Test_0

Ste

In [24]:
import carla, threading, time

def stop_manager(manager):
    manager._running = False

    t = getattr(manager, "_scenario_thread", None)
    if t and hasattr(t, "is_alive") and t.is_alive() and threading.current_thread() is not t:
        t.join(timeout=2.0)
    manager._scenario_thread = None

    # if a tick thread ever existed, stop it too (safe even if missing)
    tt = getattr(manager, "_tick_thread", None)
    if tt and hasattr(tt, "is_alive") and tt.is_alive() and threading.current_thread() is not tt:
        tt.join(timeout=2.0)
    manager._tick_thread = None

def reset_tree(tree):
    for node in tree.iterate():  # breadth-first
        try:
            node.terminate(py_trees.common.Status.INVALID)
        except Exception:
            pass
        try:
            node.initialise()
        except Exception:
            pass
    tree.status = py_trees.common.Status.RUNNING

def reinit_inroute_and_blocked(tree):
    """Re-initialise only InRouteTest and ActorBlockedTest (safe, no spawns)."""
    target = {"InRouteTest", "ActorBlockedTest"}
    for n in tree.iterate():
        if n.__class__.__name__ in target:
            try:
                n.terminate(py_trees.common.Status.INVALID)  # clear latched status
                n.initialise()                               # re-arm the check
                # print(f"reinit: {n.__class__.__name__} -> ok")  # optional
            except Exception:
                pass

def _existing_vehicle_ids(world):
    return {v.id for v in world.get_actors().filter("vehicle.*")}

def _vec3(x=0, y=0, z=0):
    return carla.Vector3D(float(x), float(y), float(z))

def start_manager_builder_only(manager):
    """Start ONLY the builder loop; NO auto-ticking."""
    manager._running = True
    t = getattr(manager, "_scenario_thread", None)
    if t is None or not (hasattr(t, "is_alive") and t.is_alive()):
        manager._scenario_thread = threading.Thread(
            target=manager.build_scenarios_loop,
            args=(manager._debug_mode > 0,),
            daemon=True
        )
        manager._scenario_thread.start()
    # <-- DO NOT call _tick_scenario() here; you will step manually.

def _to_transform(entry):
    if isinstance(entry, tuple) and len(entry) >= 1:
        entry = entry[0]

    if isinstance(entry, carla.Transform):
        return entry

    if hasattr(entry, "transform"):
        return entry.transform

    if isinstance(entry, dict):
        x = float(entry.get("x", 0.0))
        y = float(entry.get("y", 0.0))
        z = float(entry.get("z", 0.0))
        yaw = float(entry.get("yaw", 0.0))
        return carla.Transform(
            carla.Location(x, y, z),
            carla.Rotation(yaw=yaw)
        )
    raise TypeError(f"Don't know how to convert plan entry {type(entry)} to carla.Transform")

def _v3(x=0, y=0, z=0):
    return carla.Vector3D(float(x), float(y), float(z))

def snap_ego_to_route_start(manager):
    ego = manager.ego_vehicles[0]
    scn = manager.scenario

    plan = None
    if hasattr(scn, "_global_plan") and scn._global_plan:
        plan = scn._global_plan
    elif hasattr(scn, "global_plan") and scn.global_plan:
        plan = scn.global_plan
    elif hasattr(scn, "route") and scn.route:
        plan = scn.route

    if plan:
        tf = _to_transform(plan[0])
    else:
        # fallback: nearest driving waypoint from current ego loc
        wmap = manager.scenario.world.get_map()
        wp = wmap.get_waypoint(
            ego.get_location(),
            project_to_road=True,
            lane_type=carla.LaneType.Driving
        )
        tf = wp.transform

    ego.set_transform(tf)
    ego.set_target_velocity(carla.Vector3D(0,0,0))
    ego.set_target_angular_velocity(carla.Vector3D(0,0,0))

In [25]:
stop_manager(leaderboard_evaluator.manager)


In [26]:

world = leaderboard_evaluator.world

# 2) Ensure sync (paused) and remember previous settings
prev = world.get_settings()
changed = False
if not prev.synchronous_mode:
    new = carla.WorldSettings()
    new.no_rendering_mode   = prev.no_rendering_mode
    new.synchronous_mode    = True
    new.fixed_delta_seconds = prev.fixed_delta_seconds or 0.05
    world.apply_settings(new)
    changed = True

In [27]:
present = _existing_vehicle_ids(world)

In [28]:
for v in leaderboard_evaluator.manager.scenario_tree.iterate():
    # print("name:", v.name, "status:",v.status)
    pass

In [29]:
# mode = "snapshot_strict"
# apply_controls = True
# client = leaderboard_evaluator.client
# world = leaderboard_evaluator.world

# # --- MAIN RESTORE BATCH ---
# batch = []

# for rec in vehicles_state:
#     vid = int(rec["id"])
#     if vid not in present:
#         continue

#     tf = carla.Transform(
#         carla.Location(float(rec["x"]), float(rec["y"]), float(rec["z"])),
#         carla.Rotation(
#             yaw=float(rec.get("yaw", 0.0)),
#             pitch=float(rec.get("pitch", 0.0)),
#             roll=float(rec.get("roll",  0.0)),
#         )
#     )

#     # Disable physics, set transform, re-enable physics
#     batch += [
#         carla.command.SetSimulatePhysics(vid, False),
#         carla.command.ApplyTransform(vid, tf),
#         carla.command.SetSimulatePhysics(vid, True),
#     ]

# # Apply transforms first and let physics settle
# client.apply_batch_sync(batch, True)
# world.tick()  # Let physics settle after re-enabling

# # Calculate and apply impulses in smaller batches to reduce memory usage
# if mode == "snapshot_strict":
#     BATCH_SIZE = 10  # Process vehicles in groups of 10
#     vehicle_records = [(int(rec["id"]), rec) for rec in vehicles_state if int(rec["id"]) in present]

#     for i in range(0, len(vehicle_records), BATCH_SIZE):
#         impulse_batch = []
#         batch_records = vehicle_records[i:i+BATCH_SIZE]

#         for vid, rec in batch_records:
#             actor = world.get_actor(vid)
#             if not actor:
#                 continue

#             # Get mass and current velocity
#             mass = actor.get_physics_control().mass
#             v_cur = actor.get_velocity()

#             # Calculate impulse directly without creating intermediate objects
#             # J = m * (v_desired - v_current)
#             impulse_batch.append(
#                 carla.command.ApplyImpulse(
#                     vid,
#                     carla.Vector3D(
#                         mass * (rec.get("vx", 0.0) - v_cur.x),
#                         mass * (rec.get("vy", 0.0) - v_cur.y),
#                         mass * (rec.get("vz", 0.0) - v_cur.z)
#                     )
#                 )
#             )

#             # Set angular velocity if non-zero
#             wx, wy, wz = rec.get("wx", 0.0), rec.get("wy", 0.0), rec.get("wz", 0.0)
#             if wx != 0.0 or wy != 0.0 or wz != 0.0:
#                 impulse_batch.append(
#                     carla.command.ApplyTargetAngularVelocity(vid, _vec3(wx, wy, wz)))

#         # Apply this batch of impulses
#         if impulse_batch:
#             client.apply_batch_sync(impulse_batch, False)

#         # Clear the batch to free memory
#         del impulse_batch

# # Small tick to let impulses take effect
# world.tick()

# # Apply controls in batches too
# if apply_controls:
#     for i in range(0, len(vehicle_records), BATCH_SIZE):
#         control_batch = []
#         batch_records = vehicle_records[i:i+BATCH_SIZE]

#         for vid, rec in batch_records:
#             control_batch.append(
#                 carla.command.ApplyVehicleControl(
#                     vid,
#                     carla.VehicleControl(
#                         throttle=float(rec.get("throttle", 0.0)),
#                         steer=float(rec.get("steer", 0.0)),
#                         brake=float(rec.get("brake", 0.0)),
#                         hand_brake=bool(rec.get("hand_brake", False)),
#                         reverse=bool(rec.get("reverse", False)),
#                         gear=int(rec.get("gear", 0)),
#                     )
#                 )
#             )

#         if control_batch:
#             client.apply_batch_sync(control_batch, False)
#         del control_batch

# # Final tick
# world.tick()

# # Finally, reinit those two criteria only (prevents immediate FAILURE)
# reinit_inroute_and_blocked(leaderboard_evaluator.manager.scenario_tree)

In [30]:
mode = "snapshot_strict"
apply_controls = True
client = leaderboard_evaluator.client
world = leaderboard_evaluator.world

# --- MAIN RESTORE BATCH ---
batch = []

for rec in vehicles_state:
    vid = int(rec["id"])
    if vid not in present:
        continue

    tf = carla.Transform(
        carla.Location(float(rec["x"]), float(rec["y"]), float(rec["z"])),
        carla.Rotation(
            yaw=float(rec.get("yaw", 0.0)),
            pitch=float(rec.get("pitch", 0.0)),
            roll=float(rec.get("roll",  0.0)),
        )
    )

    # Disable physics, set transform, re-enable physics
    batch += [
        carla.command.SetSimulatePhysics(vid, False),
        carla.command.ApplyTransform(vid, tf),
        carla.command.SetSimulatePhysics(vid, True),
    ]

# Apply transforms first
client.apply_batch_sync(batch, True)
world.tick()  # Let physics settle

# Apply velocities using set_target_velocity directly (more reliable)
if mode == "snapshot_strict":
    for rec in vehicles_state:
        vid = int(rec["id"])
        if vid not in present:
            continue

        actor = world.get_actor(vid)
        if actor:
            # Get saved velocities
            vx = abs(rec.get("vx", 0.0))
            vy = abs(rec.get("vy", 0.0))
            vz = abs(rec.get("vz", 0.0))
            wx = rec.get("wx", 0.0)
            wy = rec.get("wy", 0.0)
            wz = rec.get("wz", 0.0)
            # print("vx, vy, vz", vx, vy, vz)

            # Debug print
            if vid == 3695:
                v_cur = actor.get_velocity()
                print(f"Ego before impulse: cur=({v_cur.x:.2f}, {v_cur.y:.2f}, {v_cur.z:.2f}), target=({vx:.2f}, {vy:.2f}, {vz:.2f})")

            # Force the exact velocity using enable_constant_velocity for one frame
            actor.enable_constant_velocity(carla.Vector3D(vx, vy, vz))
            actor.set_target_angular_velocity(carla.Vector3D(wx, wy, wz))

# Let constant velocity apply for one tick
world.tick()

# Disable constant velocity and apply controls
for rec in vehicles_state:
    vid = int(rec["id"])
    if vid not in present:
        continue

    actor = world.get_actor(vid)
    if actor:
        # Disable constant velocity
        actor.disable_constant_velocity()

        # Apply controls
        if apply_controls:
            ctrl = carla.VehicleControl(
                throttle=float(rec.get("throttle", 0.0)),
                steer=float(rec.get("steer", 0.0)),
                brake=float(rec.get("brake", 0.0)),
                hand_brake=bool(rec.get("hand_brake", False)),
                reverse=bool(rec.get("reverse", False)),
                gear=int(rec.get("gear", 0)),
            )
            actor.apply_control(ctrl)

# Final tick
world.tick()

# Finally, reinit those two criteria only (prevents immediate FAILURE)
reinit_inroute_and_blocked(leaderboard_evaluator.manager.scenario_tree)

Ego before impulse: cur=(-0.01, -0.00, -0.00), target=(12.25, 0.05, 0.01)


In [31]:
# mode = "snapshot_strict"
# apply_controls = True
# client = leaderboard_evaluator.client
# world = leaderboard_evaluator.world

# # --- MAIN RESTORE BATCH ---
# batch = []

# for rec in vehicles_state:
#     vid = int(rec["id"])
#     if vid not in present:
#         continue

#     tf = carla.Transform(
#         carla.Location(float(rec["x"]), float(rec["y"]), float(rec["z"])),
#         carla.Rotation(
#             yaw=float(rec.get("yaw", 0.0)),
#             pitch=float(rec.get("pitch", 0.0)),
#             roll=float(rec.get("roll",  0.0)),
#         )
#     )

#     vx, vy, vz = rec.get("vx", 0.0), rec.get("vy", 0.0), rec.get("vz", 0.0)
#     wx, wy, wz = rec.get("wx", 0.0), rec.get("wy", 0.0), rec.get("wz", 0.0)

#     # Apply transform
#     batch.append(carla.command.ApplyTransform(vid, tf))

#     if mode == "snapshot_strict":
#         actor = world.get_actor(vid)
#         if actor:
#             # Enable constant velocity mode to force exact velocity
#             actor.enable_constant_velocity(carla.Vector3D(vx, vy, vz))

#             # Also apply impulse for immediate effect
#             physics_control = actor.get_physics_control()
#             mass = physics_control.mass
#             impulse_x = mass * vx
#             impulse_y = mass * vy
#             impulse_z = mass * vz
#             batch.append(carla.command.ApplyImpulse(vid, carla.Vector3D(impulse_x, impulse_y, impulse_z)))

#             # Set angular velocity
#             batch.append(carla.command.ApplyTargetAngularVelocity(vid, _vec3(wx, wy, wz)))
#     else:  # continue_zero
#         batch.append(carla.command.ApplyTargetVelocity(vid, _vec3(0, 0, 0)))
#         batch.append(carla.command.ApplyTargetAngularVelocity(vid, _vec3(0, 0, 0)))

# # Apply batch
# client.apply_batch_sync(batch, True)
# world.tick()

# # Now disable constant velocity after one tick to return to normal physics
# for rec in vehicles_state:
#     vid = int(rec["id"])
#     if vid not in present:
#         continue
#     actor = world.get_actor(vid)
#     if actor:
#         actor.disable_constant_velocity()

# # Apply controls
# control_batch = []
# if apply_controls:
#     for rec in vehicles_state:
#         vid = int(rec["id"])
#         if vid not in present:
#             continue

#         control_batch.append(
#             carla.command.ApplyVehicleControl(
#                 vid,
#                 carla.VehicleControl(
#                     throttle=float(rec.get("throttle", 0.0)),
#                     steer=float(rec.get("steer", 0.0)),
#                     brake=float(rec.get("brake", 0.0)),
#                     hand_brake=bool(rec.get("hand_brake", False)),
#                     reverse=bool(rec.get("reverse", False)),
#                     gear=int(rec.get("gear", 0)),
#                 )
#             )
#         )

# if control_batch:
#     client.apply_batch_sync(control_batch, True)

# # Finally, reinit those two criteria only (prevents immediate FAILURE)
# reinit_inroute_and_blocked(leaderboard_evaluator.manager.scenario_tree)

In [32]:
actor.get_velocity().x, actor.get_velocity().y, actor.get_velocity().z

(-11.96471118927002, 0.029189791530370712, 0.014372358098626137)

In [33]:
actor.get_transform().rotation.yaw, 

(179.7603302001953,)

In [34]:
actors = world_obj.get_actors()
vehicles = actors.filter("vehicle.*")
for actor in vehicles:
    print("-----------")
    print("actor.id", actor.id)
    print("yaw:", actor.get_transform().rotation.yaw, "pitch:", actor.get_transform().rotation.pitch, "roll:", actor.get_transform().rotation.roll)
    print("speed",actor.get_velocity().x, actor.get_velocity().y, actor.get_velocity().z)

-----------
actor.id 3834
yaw: -0.2724609076976776 pitch: 0.2992168962955475 roll: -5.211091974599569e-11
speed 0.0 0.0 0.0
-----------
actor.id 3833
yaw: -0.2724609076976776 pitch: 0.2992168962955475 roll: -5.211091974599569e-11
speed 0.0 0.0 0.0
-----------
actor.id 3771
yaw: 0.06297902017831802 pitch: 0.4897245466709137 roll: 0.027929600328207016
speed 6.56464147567749 0.002352735260501504 0.03384098783135414
-----------
actor.id 3770
yaw: 0.037227582186460495 pitch: 0.22232264280319214 roll: -0.0018615720327943563
speed 6.262619972229004 0.00048146280460059643 0.02774510346353054
-----------
actor.id 3769
yaw: -179.86688232421875 pitch: 0.06436087191104889 roll: 0.005111830774694681
speed -6.241629600524902 -0.010802001692354679 0.008896112442016602
-----------
actor.id 3768
yaw: -179.95718383789062 pitch: 0.10277385264635086 roll: 0.005296168848872185
speed -8.785935401916504 -0.002156441565603018 0.007416991982609034
-----------
actor.id 3767
yaw: 179.72750854492188 pitch: -0.302

In [35]:
# -----------
# actor.id 3771
# yaw: 0.11760231107473373 pitch: 0.43278124928474426 roll: 0.016277821734547615
# speed 6.651047706604004 0.005099917761981487 0.025384750217199326
# -----------
# actor.id 3770
# yaw: 0.0877789556980133 pitch: 0.18733158707618713 roll: -9.155272709904239e-05
# speed 6.486534118652344 0.002112228889018297 0.02569270133972168
# -----------
# actor.id 3769
# yaw: -179.79873657226562 pitch: 0.07573313266038895 roll: 0.002902447013184428
# speed -6.340973854064941 -0.008854418061673641 0.006159028969705105
# -----------
# actor.id 3768
# yaw: -179.91253662109375 pitch: 0.21467965841293335 roll: 0.005242487881332636
# speed -8.929043769836426 -0.002869710326194763 0.0055673313327133656
# -----------
# actor.id 3767
# yaw: 179.72750854492188 pitch: -0.29925790429115295 roll: 0.0
# speed 0.0 0.0 0.0
# -----------
# actor.id 3766
# yaw: 179.72750854492188 pitch: -0.3584551513195038 roll: 0.0
# speed 0.0 0.0 0.0
# -----------
# actor.id 3765
# yaw: -179.77146911621094 pitch: 0.07515256851911545 roll: 0.0023206390906125307
# speed -6.326052665710449 -0.009807088412344456 0.005542411468923092
# -----------
# actor.id 3764
# yaw: -179.96168518066406 pitch: 0.24435000121593475 roll: 0.00798146054148674
# speed -9.451289176940918 -0.003668647026643157 0.005352172534912825
# -----------
# actor.id 3763
# yaw: -179.72238159179688 pitch: -0.042572565376758575 roll: -0.00012207030522404239
# speed -5.097009658813477 -0.02941063418984413 -0.001750354771502316
# -----------
# actor.id 3762
# yaw: -179.96603393554688 pitch: -0.1329086422920227 roll: 0.006951667834073305
# speed -7.8346991539001465 -0.0008160126744769514 -0.013054799288511276
# -----------
# actor.id 3697
# yaw: 179.72560119628906 pitch: -0.21338193118572235 roll: 0.019010258838534355
# speed -2.4318695068359375e-05 -0.00032338968594558537 0.009335293434560299
# -----------
# actor.id 3696
# yaw: 179.72799682617188 pitch: -0.45734262466430664 roll: 0.001633104751817882
# speed 2.413727315797587e-06 0.0003448301868047565 0.0020283746998757124
# -----------
# actor.id 3695
# yaw: 179.76072692871094 pitch: 0.7089735865592957 roll: 0.001819672528654337
# speed -12.2544527053833 0.05126480013132095 -0.008819589391350746

In [36]:
# mode = "snapshot_strict"
# apply_controls = True
# client = leaderboard_evaluator.client
# world = leaderboard_evaluator.world

# # --- MAIN RESTORE BATCH ---
# batch = []

# for rec in vehicles_state:
#     vid = int(rec["id"])
#     if vid not in present:
#         continue

#     # if vid == (3695):
#     #     print("rec", rec)

#     tf = carla.Transform(
#         carla.Location(float(rec["x"]), float(rec["y"]), float(rec["z"])),
#         carla.Rotation(
#             yaw=float(rec.get("yaw", 0.0)),
#             pitch=float(rec.get("pitch", 0.0)),
#             roll=float(rec.get("roll",  0.0)),
#         )
#     )

#     vx, vy, vz = rec.get("vx", 0.0), rec.get("vy", 0.0), rec.get("vz", 0.0)
#     wx, wy, wz = rec.get("wx", 0.0), rec.get("wy", 0.0), rec.get("wz", 0.0)

#     # Keep physics enabled and apply transform
#     batch.append(carla.command.ApplyTransform(vid, tf))

#     if mode == "snapshot_strict":
#         # Apply impulse to achieve target velocity instantly
#         actor = world.get_actor(vid)
#         if actor:
#             physics_control = actor.get_physics_control()
#             mass = physics_control.mass

#             # Calculate impulse needed to achieve target velocity
#             # Impulse = mass * velocity_change
#             current_vel = actor.get_velocity()
#             impulse_x = mass * (vx - current_vel.x)
#             impulse_y = mass * (vy - current_vel.y)
#             impulse_z = mass * (vz - current_vel.z)

#             batch.append(carla.command.ApplyImpulse(vid, carla.Vector3D(impulse_x, impulse_y, impulse_z)))

#             # Also set target velocity to maintain it
#             batch.append(carla.command.ApplyTargetVelocity(vid, _vec3(vx, vy, vz)))
#             batch.append(carla.command.ApplyTargetAngularVelocity(vid, _vec3(wx, wy, wz)))
#     else:  # continue_zero
#         batch.append(carla.command.ApplyTargetVelocity(vid, _vec3(0, 0, 0)))
#         batch.append(carla.command.ApplyTargetAngularVelocity(vid, _vec3(0, 0, 0)))

# # Apply controls
# if apply_controls:
#     for rec in vehicles_state:
#         vid = int(rec["id"])
#         if vid not in present:
#             continue

#         batch.append(
#             carla.command.ApplyVehicleControl(
#                 vid,
#                 carla.VehicleControl(
#                     throttle=float(rec.get("throttle", 0.0)),
#                     steer=float(rec.get("steer", 0.0)),
#                     brake=float(rec.get("brake", 0.0)),
#                     hand_brake=bool(rec.get("hand_brake", False)),
#                     reverse=bool(rec.get("reverse", False)),
#                     gear=int(rec.get("gear", 0)),
#                 )
#             )
#         )

# # Apply everything in one batch
# client.apply_batch_sync(batch, True)
# world.tick()

# # Finally, reinit those two criteria only (prevents immediate FAILURE)
# reinit_inroute_and_blocked(leaderboard_evaluator.manager.scenario_tree)

In [37]:
# actor.get_velocity().x, actor.get_velocity().y, actor.get_velocity().z

In [38]:
# set_speed_ego = carla.Vector3D(vehicles_state[-1]['vx'], vehicles_state[-1]['vy'], vehicles_state[-1]['vz'])

In [39]:
# actor.set_target_velocity(set_speed_ego)
# leaderboard_evaluator.world.tick()

In [40]:
# actor.enable_constant_velocity(set_speed_ego)
# leaderboard_evaluator.world.tick()

In [41]:
# mode = "snapshot_strict"
# apply_controls = True
# client = leaderboard_evaluator.client
# world = leaderboard_evaluator.world

# # --- MAIN RESTORE BATCH ---
# batch = []
# post_ctrl = []  # collect controls to send AFTER velocities are set

# for rec in vehicles_state:
#     vid = int(rec["id"])
#     if vid not in present:
#         continue

#     tf = carla.Transform(
#         carla.Location(float(rec["x"]), float(rec["y"]), float(rec["z"])),
#         carla.Rotation(
#             yaw=float(rec.get("yaw", 0.0)),
#             pitch=float(rec.get("pitch", 0.0)),
#             roll=float(rec.get("roll",  0.0)),
#         )
#     )

#     vx, vy, vz = rec.get("vx", 0.0), rec.get("vy", 0.0), rec.get("vz", 0.0)
#     wx, wy, wz = rec.get("wx", 0.0), rec.get("wy", 0.0), rec.get("wz", 0.0)

#     # Apply transform with physics disabled
#     batch += [
#         carla.command.SetSimulatePhysics(vid, False),
#         carla.command.ApplyTransform(vid, tf),
#     ]

# # First batch: position all vehicles with physics OFF
# client.apply_batch_sync(batch, False)

# # Now set velocities directly on each actor and re-enable physics
# for rec in vehicles_state:
#     vid = int(rec["id"])
#     if vid not in present:
#         continue

#     actor = world.get_actor(vid)
#     if actor:
#         vx, vy, vz = rec.get("vx", 0.0), rec.get("vy", 0.0), rec.get("vz", 0.0)
#         wx, wy, wz = rec.get("wx", 0.0), rec.get("wy", 0.0), rec.get("wz", 0.0)

#         if mode == "snapshot_strict":
#             # Set velocity DIRECTLY (not target velocity)
#             actor.set_velocity(carla.Vector3D(vx, vy, vz))
#             actor.set_angular_velocity(carla.Vector3D(wx, wy, wz))
#         else:  # continue_zero
#             actor.set_velocity(carla.Vector3D(0, 0, 0))
#             actor.set_angular_velocity(carla.Vector3D(0, 0, 0))

#         # Re-enable physics
#         actor.set_simulate_physics(True)

# # Tick once to let velocities settle
# world.tick()

# # Apply recorded controls (throttle/steer/brake/etc.) now
# if apply_controls:
#     control_batch = []
#     for rec in vehicles_state:
#         vid = int(rec["id"])
#         if vid not in present:
#             continue

#         control_batch.append(
#             carla.command.ApplyVehicleControl(
#                 vid,
#                 carla.VehicleControl(
#                     throttle=float(rec.get("throttle", 0.0)),
#                     steer=float(rec.get("steer", 0.0)),
#                     brake=float(rec.get("brake", 0.0)),
#                     hand_brake=bool(rec.get("hand_brake", False)),
#                     reverse=bool(rec.get("reverse", False)),
#                     gear=int(rec.get("gear", 0)),
#                 )
#             )
#         )

#     if control_batch:
#         client.apply_batch_sync(control_batch, True)

# # Finally, reinit those two criteria only (prevents immediate FAILURE)
# reinit_inroute_and_blocked(leaderboard_evaluator.manager.scenario_tree)

# # # --- MAIN RESTORE BATCH ---
# # batch = []
# # post_ctrl = []  # collect controls to send AFTER velocities are set

# # for rec in vehicles_state:
# #     vid = int(rec["id"])
# #     if vid not in present:
# #         continue

# #     tf = carla.Transform(
# #         carla.Location(float(rec["x"]), float(rec["y"]), float(rec["z"])),
# #         carla.Rotation(
# #             yaw=float(rec.get("yaw", 0.0)),
# #             pitch=float(rec.get("pitch", 0.0)),
# #             roll=float(rec.get("roll",  0.0)),
# #         )
# #     )

# #     vx, vy, vz = rec.get("vx", 0.0), rec.get("vy", 0.0), rec.get("vz", 0.0)
# #     wx, wy, wz = rec.get("wx", 0.0), rec.get("wy", 0.0), rec.get("wz", 0.0)

# #     # Correct order: physics OFF → transform → physics ON → set v/ω
# #     batch += [
# #         carla.command.SetSimulatePhysics(vid, False),
# #         carla.command.ApplyTransform(vid, tf),
# #         carla.command.SetSimulatePhysics(vid, True),
# #     ]

# #     if mode == "snapshot_strict":
# #         batch += [
# #             carla.command.ApplyTargetVelocity(vid, _vec3(vx, vy, vz)),
# #             carla.command.ApplyTargetAngularVelocity(vid, _vec3(wx, wy, wz)),
# #         ]
# #     else:  # continue_zero
# #         batch += [
# #             carla.command.ApplyTargetVelocity(vid, _vec3(0, 0, 0)),
# #             carla.command.ApplyTargetAngularVelocity(vid, _vec3(0, 0, 0)),
# #         ]

# #     # Collect control to apply AFTER one settling tick (so your v/ω stick for at least one frame)
# #     if apply_controls:
# #         post_ctrl.append(
# #             carla.command.ApplyVehicleControl(
# #                 vid,
# #                 carla.VehicleControl(
# #                     throttle=float(rec.get("throttle", 0.0)),
# #                     steer=float(rec.get("steer", 0.0)),
# #                     brake=float(rec.get("brake", 0.0)),
# #                     hand_brake=bool(rec.get("hand_brake", False)),
# #                     reverse=bool(rec.get("reverse", False)),
# #                     gear=int(rec.get("gear", 0)),
# #                 )
# #             )
# #         )

# # # Apply atomically and let physics take one or two steps to settle
# # client.apply_batch_sync(batch, True)  # do_tick=True
# # world.tick()  # one settle tick so velocities apply before controls

# # # Apply recorded controls (throttle/steer/brake/etc.) now
# # if apply_controls and post_ctrl:
# #     client.apply_batch_sync(post_ctrl, True)
# #     # optional extra settle
# #     world.tick()

# # # If you want TM to roughly keep snapshot speeds, you can try:
# # # for rec in vehicles_state:
# # #     actor = world.get_actor(int(rec["id"]))
# # #     if actor:
# # #         speed = float((rec.get("vx",0.0)**2 + rec.get("vy",0.0)**2 + rec.get("vz",0.0)**2)**0.5)
# # #         try: tm.set_desired_speed(actor, speed)
# # #         except: pass

# # # Finally, reinit those two criteria only (prevents immediate FAILURE)
# # reinit_inroute_and_blocked(leaderboard_evaluator.manager.scenario_tree)

In [42]:
# mode = "snapshot_strict"
# apply_controls = True
# client = leaderboard_evaluator.client
# world = leaderboard_evaluator.world

# # restore_vehicles(world, vehicles_state)

# batch = []
# for rec in vehicles_state:
#     vid = int(rec["id"])
#     if vid not in present:
#         continue

#     tf = carla.Transform(
#         carla.Location(float(rec["x"]), float(rec["y"]), float(rec["z"])),
#         carla.Rotation(
#             yaw=float(rec.get("yaw", 0.0)),
#             pitch=float(rec.get("pitch", 0.0)),
#             roll=float(rec.get("roll", 0.0)),
#         )
#     )

#     vx, vy, vz = rec.get("vx", 0.0), rec.get("vy", 0.0), rec.get("vz", 0.0)
#     wx, wy, wz = rec.get("wx", 0.0), rec.get("wy", 0.0), rec.get("wz", 0.0)

#     # Correct order: physics OFF → transform → physics ON → set v/ω → (optional) control
#     batch += [
#         carla.command.SetSimulatePhysics(vid, False),
#         carla.command.ApplyTransform(vid, tf),
#         carla.command.SetSimulatePhysics(vid, True),
#     ]

#     if mode == "snapshot_strict":
#         batch += [
#             carla.command.ApplyTargetVelocity(vid, _vec3(vx, vy, vz)),
#             carla.command.ApplyTargetAngularVelocity(vid, _vec3(wx, wy, wz)),
#         ]
#     else:  # continue_zero
#         batch += [
#             carla.command.ApplyTargetVelocity(vid, _vec3(0, 0, 0)),
#             carla.command.ApplyTargetAngularVelocity(vid, _vec3(0, 0, 0)),
#         ]

#     if apply_controls:
#         ctrl = carla.VehicleControl(
#             throttle=float(rec.get("throttle", 0.0)),
#             steer=float(rec.get("steer", 0.0)),
#             brake=float(rec.get("brake", 0.0)),
#             hand_brake=bool(rec.get("hand_brake", False)),
#             reverse=bool(rec.get("reverse", False)),
#             gear=int(rec.get("gear", 0)),
#         )
#         batch.append(carla.command.ApplyVehicleControl(vid, ctrl))

# # Apply atomically and let physics take one or two steps to settle
# client.apply_batch_sync(batch, True)  # do_tick=True
# world.tick(); world.tick()

# reinit_inroute_and_blocked(leaderboard_evaluator.manager.scenario_tree)

In [43]:
ego = world.get_actor(3695)  # your ego id
print("ego v len:", ego.get_velocity().length(),
      "ω:", ego.get_angular_velocity())

ego v len: 11.96475601196289 ω: Vector3D(x=0.215502, y=-1.794861, z=-0.004513)


In [44]:
# 7) Restore previous async mode (only if requested)
if changed and not keep_sync:
    world.apply_settings(prev)

start_manager_builder_only(leaderboard_evaluator.manager)

In [45]:
### third 50s 
# Create save directory with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
save_dir = Path(f"/mnt3/Documents/AD_Framework/bench2drive-gymnasium/bench2drive_microservices/notebooks/debug_snapshots/debug_images_{timestamp}")
save_dir.mkdir(parents=True, exist_ok=True)
print(f"Images will be saved to: {save_dir}")

run_step_n = 50  # run 5 steps = 0.25 second
total_steps_run = 0

if not crash_flag:
    print(f"\n=== Running {run_step_n} simulation steps with image capture ===")
    
    for step_i in range(run_step_n):
        # Run one step
        crash_flag_step, entry_status, crash_message = my_run_scenario_step(
            leaderboard_evaluator, entry_status, crash_message, n_steps=1
        )
        
        if crash_flag_step:
            crash_flag = True
            print(f"Crash detected at step {step_i}")
            break
        
        # Save images after each step
        if hasattr(leaderboard_evaluator.agent_instance, 'last_input_data'):
            print(f"\nStep {total_steps_run + step_i}:")
            step_dir = save_debug_images(
                leaderboard_evaluator.agent_instance, 
                save_dir, 
                total_steps_run + step_i
            )
        else:
            print(f"Step {total_steps_run + step_i}: No sensor data captured")
    
    total_steps_run += step_i + 1
    
print(f"\n=== Completed {total_steps_run} steps ===")
print(f"crash_flag: {crash_flag}")
print(f"entry_status: {entry_status}")
print(f"crash_message: {crash_message}")
print(f"Images saved to: {save_dir}")

Images will be saved to: /mnt3/Documents/AD_Framework/bench2drive-gymnasium/bench2drive_microservices/notebooks/debug_snapshots/debug_images_20250903_023424

=== Running 50 simulation steps with image capture ===
In my_run_scenario_step (Check if it is stuck or not)
=== [Agent] -- Wallclock = 2025-09-03 02:34:24.252 -- System time = 27.622 -- Game time = 5.250 -- Ratio = 0.190x
Frame 101/10 - Test_0

Step 0:
In my_run_scenario_step (Check if it is stuck or not)
=== [Agent] -- Wallclock = 2025-09-03 02:34:24.452 -- System time = 27.823 -- Game time = 5.300 -- Ratio = 0.190x
Frame 102/10 - Test_0

Step 1:
In my_run_scenario_step (Check if it is stuck or not)
=== [Agent] -- Wallclock = 2025-09-03 02:34:24.698 -- System time = 28.068 -- Game time = 5.350 -- Ratio = 0.191x
Frame 103/10 - Test_0

Step 2:
In my_run_scenario_step (Check if it is stuck or not)
=== [Agent] -- Wallclock = 2025-09-03 02:34:24.909 -- System time = 28.279 -- Game time = 5.400 -- Ratio = 0.191x
Frame 104/10 - Test_0


In [46]:
for v in leaderboard_evaluator.manager.scenario_tree.iterate():
    print("name:", v.name, "status:",v.status)


name: LightsBehavior status: Status.RUNNING
name: RouteWeatherBehavior status: Status.RUNNING
name: ScenarioTriggerer status: Status.RUNNING
name: BackgroundActivity status: Status.RUNNING
name: WaitForBlackboardVariable: ScenarioRouteNumber0 status: Status.SUCCESS
name: LeaveSpaceInFront status: Status.SUCCESS
name: ChangeRoadBehavior status: Status.SUCCESS
name: ScenarioTimeout status: Status.RUNNING
name: WaitUntilInFront status: Status.RUNNING
name: DriveDistance status: Status.INVALID
name: End Condition status: Status.RUNNING
name: BasicAgentBehavior status: Status.RUNNING
name: Braking status: Status.INVALID
name: WaitForever status: Status.INVALID
name: Bicycle behavior status: Status.RUNNING
name: BasicAgentBehavior status: Status.RUNNING
name: Braking status: Status.INVALID
name: WaitForever status: Status.INVALID
name: Bicycle behavior status: Status.RUNNING
name: TriggerDistanceToVehicle status: Status.SUCCESS
name: Idle status: Status.SUCCESS
name: SetMaxSpeed status: Stat

In [ ]:
# mass = actor.get_physics_control().mass

# v_cur = actor.get_velocity()

# print("m", m)
# print("v_cur", v_cur)

# ## record (save)
# v_des = carla.Vector3D(rec["vx"], rec["vy"], rec["vz"])

# # difference
# dv = carla.Vector3D(v_des.x - v_cur.x,
#                     v_des.y - v_cur.y,
#                     v_des.z - v_cur.z)

# # impulse J = m * Δv   (units: N·s)
# J = carla.Vector3D(mass * dv.x,
#                    mass * dv.y,
#                    mass * dv.z)


In [ ]:

def pause_restore_resume(client, world, manager, vehicles_state, mode="snapshot_strict",      # "snapshot_strict" or "scenario_continue"
    apply_controls=True,         # reapply VehicleControl if present in rec
    keep_sync=True               # stay in sync mode after restore
):
    """
    vehicles_state: iterable of dicts, each with at least:
      id,x,y,z[,yaw,pitch,roll,vx,vy,vz,wx,wy,wz,throttle,steer,brake,hand_brake,reverse,gear]
    """

    # 1) Pause any manager activity
    stop_manager(manager)

    # 2) Ensure sync (paused) and remember previous settings
    prev = world.get_settings()
    changed = False
    if not prev.synchronous_mode:
        new = carla.WorldSettings()
        new.no_rendering_mode   = prev.no_rendering_mode
        new.synchronous_mode    = True
        new.fixed_delta_seconds = prev.fixed_delta_seconds or 0.05
        world.apply_settings(new)
        changed = True

    # 3) Build atomic batch
    present = _existing_vehicle_ids(world)
    batch = []
    for rec in vehicles_state:
        vid = int(rec["id"])
        if vid not in present:
            # If you have role_name in rec, you could map to the current id here.
            continue

        tf = carla.Transform(
            carla.Location(float(rec["x"]), float(rec["y"]), float(rec["z"])),
            carla.Rotation(
                yaw=float(rec.get("yaw",   0.0)),
                pitch=float(rec.get("pitch", 0.0)),
                roll=float(rec.get("roll",  0.0)),
            )
        )

        # velocities (default to 0 if not captured)
        vx, vy, vz = rec.get("vx", 0.0), rec.get("vy", 0.0), rec.get("vz", 0.0)
        wx, wy, wz = rec.get("wx", 0.0), rec.get("wy", 0.0), rec.get("wz", 0.0)

        # sequence:
        #   physics off -> transform -> (strict: restore v/w; continue: zero v/w) -> physics on -> (optional) control
        batch.append(carla.command.SetSimulatePhysics(vid, False))
        batch.append(carla.command.ApplyTransform(vid, tf))

        if mode == "snapshot_strict":
            batch.append(carla.command.ApplyTargetVelocity(vid, _v3(vx, vy, vz)))
            batch.append(carla.command.ApplyTargetAngularVelocity(vid, _v3(wx, wy, wz)))
        else:
            batch.append(carla.command.ApplyTargetVelocity(vid, _v3(0, 0, 0)))
            batch.append(carla.command.ApplyTargetAngularVelocity(vid, _v3(0, 0, 0)))

        batch.append(carla.command.SetSimulatePhysics(vid, True))

        if apply_controls:
            ctrl = carla.VehicleControl(
                throttle=float(rec.get("throttle", 0.0)),
                steer=float(rec.get("steer", 0.0)),
                brake=float(rec.get("brake", 0.0)),
                hand_brake=bool(rec.get("hand_brake", False)),
                reverse=bool(rec.get("reverse", False)),
                gear=int(rec.get("gear", 0)),
            )
            batch.append(carla.command.ApplyVehicleControl(vid, ctrl))

    # 4) Apply atomically and settle one or two frames
    resps = client.apply_batch_sync(batch, True)  # True == do_tick in 0.9.15
    # (optional) inspect errors:
    # errs = [r.error for r in resps if r.error];  print("batch errors:", errs)
    world.tick(); world.tick()

    # 5) Reset blackboard keys your tree expects (optional)
    try:
        import py_trees
        py_trees.blackboard.Blackboard().set("ScenarioRouteNumber0", None, overwrite=True)
    except Exception:
        pass

    # 6) Reset the py_trees statuses
    reset_tree(manager.scenario_tree)

    # 7) Restore previous async mode (only if requested)
    if changed and not keep_sync:
        world.apply_settings(prev)

    # 8) Resume ONLY the builder if you’re in scenario_continue. For strict snapshot,
    #    leave it off until you manually step & verify.
    if mode == "scenario_continue":
        start_manager_builder_only(manager)

In [ ]:

# set_carla_loc_rot(carla_vehicle_object=leaderboard_evaluator.manager.scenario.list_scenarios[0].other_actors[0], x=568.29, y=3914, z=371.2144, yaw=179.72, pitch=-0.458, roll=0)
# set_carla_loc_rot(carla_vehicle_object=leaderboard_evaluator.manager.ego_vehicles[0], x=578.208984375, y=3910, z=371.29, yaw=179.76, pitch=0.708, roll=0)

In [ ]:
# world_obj = leaderboard_evaluator.manager.scenario.world

In [ ]:
# import json
# import csv     

# world_obj = leaderboard_evaluator.manager.scenario.world

# def record_vehicles(world_obj, save_path=None, save_json=False):
#     actors = world_obj.get_actors()
#     vehicles = actors.filter("vehicle.*")

#     records = []
#     for v in vehicles:
#         tf = v.get_transform()
#         loc, rot = tf.location, tf.rotation
#         vel = v.get_velocity()
#         ang = v.get_angular_velocity()
#         acc = v.get_acceleration()
#         ctrl = v.get_control()

#         record = {
#             "id": v.id,
#             "type_id": v.type_id,
#             "x": loc.x,
#             "y": loc.y,
#             "z": loc.z,
#             "pitch": rot.pitch,
#             "yaw": rot.yaw,
#             "roll": rot.roll,
#             "vx": vel.x, "vy": vel.y, "vz": vel.z,
#             "wx": ang.x, "wy": ang.y, "wz": ang.z,
#             "ax": acc.x, "ay": acc.y, "az": acc.z,
#             "throttle": ctrl.throttle,
#             "steer": ctrl.steer,
#             "brake": ctrl.brake,
#             "hand_brake": ctrl.hand_brake,
#             "reverse": ctrl.reverse,
#             "gear": ctrl.gear,
#         }
#         records.append(record)

#     if save_path:
#         if save_json:
#             import json
#             with open(save_path, "w") as f:
#                 json.dump(records, f, indent=4)
#             print(f"Saved {len(records)} vehicle records to {save_path} (JSON)")
#         else:
#             import csv
#             with open(save_path, mode="w", newline="") as f:
#                 writer = csv.DictWriter(f, fieldnames=records[0].keys())
#                 writer.writeheader()
#                 writer.writerows(records)
#             print(f"Saved {len(records)} vehicle records to {save_path} (CSV)")

#     return records

# def record_ego_vehicle(route_scenario):
#     v = route_scenario.ego_vehicles[0]
#     tf = v.get_transform()
#     loc, rot = tf.location, tf.rotation
#     record = {
#         "id": v.id,
#         "type_id": v.type_id,
#         "x": loc.x,
#         "y": loc.y,
#         "z": loc.z,
#         "pitch": rot.pitch,
#         "yaw": rot.yaw,
#         "roll": rot.roll
#     }
#     return record

# def record_scenario_vehicle(route_scenario):
#     other_actor_objects = route_scenario.other_actors

#     records = []
#     for v in other_actor_objects:
#         tf = v.get_transform()
#         loc, rot = tf.location, tf.rotation
#         record = {
#             "id": v.id,
#             "type_id": v.type_id,
#             "x": loc.x,
#             "y": loc.y,
#             "z": loc.z,
#             "pitch": rot.pitch,
#             "yaw": rot.yaw,
#             "roll": rot.roll
#         }
#         records.append(record)
    
#     return records

# def watchdog_save_state(watchdog_object):
#     record_dict = {
#         "timeout": watchdog_object._timeout,
#         "interval": watchdog_object._interval,
#         "failed": watchdog_object._failed,
#         "stopped": watchdog_object._watchdog_stopped,
#     }
#     return record_dict

# ### to save state 
# vehicles_state = record_vehicles(world_obj)
# print("vehicles_state", vehicles_state)
# print("--------------------------------")

# # ego_vehicle_state = record_ego_vehicle(leaderboard_evaluator.manager.scenario)
# # print("ego_vehicle_state", ego_vehicle_state)
# # print("--------------------------------")

# # scenario_vehicle_state = record_scenario_vehicle(leaderboard_evaluator.manager.scenario.list_scenarios[0])
# # print("scenario_vehicle_state", scenario_vehicle_state)
# # print("--------------------------------")

# watchdog_state = watchdog_save_state(leaderboard_evaluator.manager._watchdog)
# print("watchdog_state", watchdog_state)
# print("--------------------------------")

In [ ]:
import carla, threading, time

def stop_manager(manager):
    manager._running = False

    t = getattr(manager, "_scenario_thread", None)
    if t and hasattr(t, "is_alive") and t.is_alive() and threading.current_thread() is not t:
        t.join(timeout=2.0)
    manager._scenario_thread = None

    # if a tick thread ever existed, stop it too (safe even if missing)
    tt = getattr(manager, "_tick_thread", None)
    if tt and hasattr(tt, "is_alive") and tt.is_alive() and threading.current_thread() is not tt:
        tt.join(timeout=2.0)
    manager._tick_thread = None

def reset_tree(tree):
    for node in tree.iterate():  # breadth-first
        try:
            node.terminate(py_trees.common.Status.INVALID)
        except Exception:
            pass
        try:
            node.initialise()
        except Exception:
            pass
    tree.status = py_trees.common.Status.RUNNING

def _existing_vehicle_ids(world):
    return {v.id for v in world.get_actors().filter("vehicle.*")}

def _vec3(x=0, y=0, z=0):
    return carla.Vector3D(float(x), float(y), float(z))

def start_manager_builder_only(manager):
    """Start ONLY the builder loop; NO auto-ticking."""
    manager._running = True
    t = getattr(manager, "_scenario_thread", None)
    if t is None or not (hasattr(t, "is_alive") and t.is_alive()):
        manager._scenario_thread = threading.Thread(
            target=manager.build_scenarios_loop,
            args=(manager._debug_mode > 0,),
            daemon=True
        )
        manager._scenario_thread.start()
    # <-- DO NOT call _tick_scenario() here; you will step manually.

def _to_transform(entry):
    if isinstance(entry, tuple) and len(entry) >= 1:
        entry = entry[0]

    if isinstance(entry, carla.Transform):
        return entry

    if hasattr(entry, "transform"):
        return entry.transform

    if isinstance(entry, dict):
        x = float(entry.get("x", 0.0))
        y = float(entry.get("y", 0.0))
        z = float(entry.get("z", 0.0))
        yaw = float(entry.get("yaw", 0.0))
        return carla.Transform(
            carla.Location(x, y, z),
            carla.Rotation(yaw=yaw)
        )
    raise TypeError(f"Don't know how to convert plan entry {type(entry)} to carla.Transform")

def _v3(x=0, y=0, z=0):
    return carla.Vector3D(float(x), float(y), float(z))

def snap_ego_to_route_start(manager):
    ego = manager.ego_vehicles[0]
    scn = manager.scenario

    plan = None
    if hasattr(scn, "_global_plan") and scn._global_plan:
        plan = scn._global_plan
    elif hasattr(scn, "global_plan") and scn.global_plan:
        plan = scn.global_plan
    elif hasattr(scn, "route") and scn.route:
        plan = scn.route

    if plan:
        tf = _to_transform(plan[0])
    else:
        # fallback: nearest driving waypoint from current ego loc
        wmap = manager.scenario.world.get_map()
        wp = wmap.get_waypoint(
            ego.get_location(),
            project_to_road=True,
            lane_type=carla.LaneType.Driving
        )
        tf = wp.transform

    ego.set_transform(tf)
    ego.set_target_velocity(carla.Vector3D(0,0,0))
    ego.set_target_angular_velocity(carla.Vector3D(0,0,0))

# def pause_restore_resume(client, world, manager, vehicles_state, mode="snapshot_strict", keep_sync=True):
#     stop_manager(manager)

#     prev = world.get_settings()
#     changed = False
#     if not prev.synchronous_mode:
#         new = carla.WorldSettings()
#         new.no_rendering_mode   = prev.no_rendering_mode
#         new.synchronous_mode    = True
#         new.fixed_delta_seconds = prev.fixed_delta_seconds or 0.05
#         world.apply_settings(new)
#         changed = True

#     present = _existing_vehicle_ids(world)

#     batch = []
#     restored = 0
#     for rec in vehicles_state:
#         vid = int(rec["id"])
#         if vid not in present:
#             # skip missing; you can print/log this if you want
#             # print(f"[restore] skip missing actor id {vid}")
#             continue

#         tf = carla.Transform(
#             carla.Location(float(rec["x"]), float(rec["y"]), float(rec["z"])),
#             carla.Rotation(
#                 yaw=float(rec.get("yaw",   0.0)),
#                 pitch=float(rec.get("pitch", 0.0)),
#                 roll=float(rec.get("roll",  0.0)),
#             )
#         )

#         if mode == "snapshot_strict":
#             # Reapply captured velocities if available; else use zeros
#             vx, vy, vz = rec.get("vx", 0.0), rec.get("vy", 0.0), rec.get("vz", 0.0)
#             wx, wy, wz = rec.get("wx", 0.0), rec.get("wy", 0.0), rec.get("wz", 0.0)

#             batch += [
#                 carla.command.SetSimulatePhysics(vid, False),
#                 carla.command.ApplyTransform(vid, tf),
#                 carla.command.ApplyTargetVelocity(vid, _vec3(vx, vy, vz)),
#                 carla.command.ApplyTargetAngularVelocity(vid, _vec3(wx, wy, wz)),
#                 carla.command.SetSimulatePhysics(vid, True),
#             ]
#         else:
#             batch += [
#                 carla.command.SetSimulatePhysics(vid, False),
#                 carla.command.ApplyTransform(vid, tf),
#                 carla.command.ApplyTargetVelocity(vid, _vec3(0,0,0)),
#                 carla.command.ApplyTargetAngularVelocity(vid, _vec3(0,0,0)),
#                 carla.command.SetSimulatePhysics(vid, True),
#             ]
#         restored += 1

#     resps = client.apply_batch_sync(batch, True)  # do_tick
#     # Optional: surface failures
#     # fails = [r for r in resps if r.error]
#     # if fails: print("Batch apply errors:", [r.error for r in fails])

#     # Minimal settling
#     world.tick()

#     # Ego handling
#     if mode == "scenario_continue":
#         # Put ego back on route (not from snapshot)
#         snap_ego_to_route_start(manager)
#         world.tick()

#     # Blackboard & tree reset
#     try:
#         py_trees.blackboard.Blackboard().set("ScenarioRouteNumber0", None, overwrite=True)
#     except Exception:
#         pass

#     reset_tree(manager.scenario_tree)

#     if changed and not keep_sync:
#         world.apply_settings(prev)

#     start_manager_builder_only(manager)

    # In strict mode, do NOT start builder yet (it will move stuff).
    # You will step manually; start builder only if you want background logic to resume.
    # if mode == "scenario_continue":
    #     start_manager_builder_only(manager)
    # For visibility
    # print(f"[restore] restored {restored} actors; mode={mode}")


# def pause_restore_resume(client, world, manager, vehicles_state, keep_sync=True):
#     stop_manager(manager)

#     prev = world.get_settings()
#     changed = False
#     if not prev.synchronous_mode:
#         new = carla.WorldSettings()
#         new.no_rendering_mode   = prev.no_rendering_mode
#         new.synchronous_mode    = True
#         new.fixed_delta_seconds = prev.fixed_delta_seconds or 0.05
#         world.apply_settings(new)
#         changed = True

#     # Build batch for all vehicles (NPCs + ego). If you prefer to exclude ego,
#     # filter vehicles_state by id != ego.id instead.
#     batch = []
#     for rec in vehicles_state:
#         vid = int(rec["id"])
#         tf  = carla.Transform(
#             carla.Location(float(rec["x"]), float(rec["y"]), float(rec["z"])),
#             carla.Rotation(
#                 yaw=float(rec.get("yaw",   0.0)),
#                 pitch=float(rec.get("pitch", 0.0)),
#                 roll=float(rec.get("roll",  0.0))
#             )
#         )
#         batch += [
#             carla.command.SetSimulatePhysics(vid, False),
#             carla.command.ApplyTransform(vid, tf),
#             carla.command.ApplyTargetVelocity(vid, carla.Vector3D(0,0,0)),
#             carla.command.ApplyTargetAngularVelocity(vid, carla.Vector3D(0,0,0)),
#             carla.command.SetSimulatePhysics(vid, True),
#         ]

#     # Apply batch & settle
#     client.apply_batch_sync(batch, True)
#     world.tick(); world.tick()

#     # NOW snap ego onto route so InRouteTest doesn't fail
#     # snap_ego_to_route_start(manager)
#     # world.tick(); world.tick()

#     # Reset blackboard key if your tree uses it (optional but often needed)
#     try:
#         py_trees.blackboard.Blackboard().set("ScenarioRouteNumber0", None, overwrite=True)
#     except Exception:
#         pass

#     # Reset the py_trees statuses
#     reset_tree(manager.scenario_tree)

#     if changed and not keep_sync:
#         world.apply_settings(prev)

#     start_manager_builder_only(manager)

def pause_restore_resume(client, world, manager, vehicles_state, mode="snapshot_strict",      # "snapshot_strict" or "scenario_continue"
    apply_controls=True,         # reapply VehicleControl if present in rec
    keep_sync=True               # stay in sync mode after restore
):
    """
    vehicles_state: iterable of dicts, each with at least:
      id,x,y,z[,yaw,pitch,roll,vx,vy,vz,wx,wy,wz,throttle,steer,brake,hand_brake,reverse,gear]
    """

    # 1) Pause any manager activity
    stop_manager(manager)

    # 2) Ensure sync (paused) and remember previous settings
    prev = world.get_settings()
    changed = False
    if not prev.synchronous_mode:
        new = carla.WorldSettings()
        new.no_rendering_mode   = prev.no_rendering_mode
        new.synchronous_mode    = True
        new.fixed_delta_seconds = prev.fixed_delta_seconds or 0.05
        world.apply_settings(new)
        changed = True

    # 3) Build atomic batch
    present = _existing_vehicle_ids(world)
    batch = []
    for rec in vehicles_state:
        vid = int(rec["id"])
        if vid not in present:
            # If you have role_name in rec, you could map to the current id here.
            continue

        tf = carla.Transform(
            carla.Location(float(rec["x"]), float(rec["y"]), float(rec["z"])),
            carla.Rotation(
                yaw=float(rec.get("yaw",   0.0)),
                pitch=float(rec.get("pitch", 0.0)),
                roll=float(rec.get("roll",  0.0)),
            )
        )

        # velocities (default to 0 if not captured)
        vx, vy, vz = rec.get("vx", 0.0), rec.get("vy", 0.0), rec.get("vz", 0.0)
        wx, wy, wz = rec.get("wx", 0.0), rec.get("wy", 0.0), rec.get("wz", 0.0)

        # sequence:
        #   physics off -> transform -> (strict: restore v/w; continue: zero v/w) -> physics on -> (optional) control
        batch.append(carla.command.SetSimulatePhysics(vid, False))
        batch.append(carla.command.ApplyTransform(vid, tf))

        if mode == "snapshot_strict":
            batch.append(carla.command.ApplyTargetVelocity(vid, _v3(vx, vy, vz)))
            batch.append(carla.command.ApplyTargetAngularVelocity(vid, _v3(wx, wy, wz)))
        else:
            batch.append(carla.command.ApplyTargetVelocity(vid, _v3(0, 0, 0)))
            batch.append(carla.command.ApplyTargetAngularVelocity(vid, _v3(0, 0, 0)))

        batch.append(carla.command.SetSimulatePhysics(vid, True))

        if apply_controls:
            ctrl = carla.VehicleControl(
                throttle=float(rec.get("throttle", 0.0)),
                steer=float(rec.get("steer", 0.0)),
                brake=float(rec.get("brake", 0.0)),
                hand_brake=bool(rec.get("hand_brake", False)),
                reverse=bool(rec.get("reverse", False)),
                gear=int(rec.get("gear", 0)),
            )
            batch.append(carla.command.ApplyVehicleControl(vid, ctrl))

    # 4) Apply atomically and settle one or two frames
    resps = client.apply_batch_sync(batch, True)  # True == do_tick in 0.9.15
    # (optional) inspect errors:
    # errs = [r.error for r in resps if r.error];  print("batch errors:", errs)
    world.tick(); world.tick()

    # 5) Reset blackboard keys your tree expects (optional)
    try:
        import py_trees
        py_trees.blackboard.Blackboard().set("ScenarioRouteNumber0", None, overwrite=True)
    except Exception:
        pass

    # 6) Reset the py_trees statuses
    reset_tree(manager.scenario_tree)

    # 7) Restore previous async mode (only if requested)
    if changed and not keep_sync:
        world.apply_settings(prev)

    # 8) Resume ONLY the builder if you’re in scenario_continue. For strict snapshot,
    #    leave it off until you manually step & verify.
    if mode == "scenario_continue":
        start_manager_builder_only(manager)

In [ ]:
pause_restore_resume(
    leaderboard_evaluator.client,
    leaderboard_evaluator.world,
    leaderboard_evaluator.manager,
    vehicles_state,
    mode="snapshot_strict",
    apply_controls=True,
    keep_sync=True,
)

In [ ]:
# Example usage
from numpy import True_


# reset_tree(leaderboard_evaluator.manager.scenario_tree)

pause_restore_resume(
    leaderboard_evaluator.client,
    leaderboard_evaluator.manager.scenario.world,
    leaderboard_evaluator.manager,
    vehicles_state,
    keep_sync=True,   # stay in sync mode for deterministic manual stepping
)

# Now advance manually
# done, entry_status, crash_msg = my_run_scenario_step(
#     leaderboard_evaluator,
#     entry_status="Started",
#     crash_message="",
#     n_steps=50
# )


In [ ]:
# leaderboard_evaluator.manager._running = True

m = leaderboard_evaluator.manager
print("running:", m._running)
print("builder alive:", getattr(m, "_scenario_thread", None) and m._scenario_thread.is_alive())
# no tick thread expected
print("tick thread exists:", hasattr(m, "_tick_thread") and m._tick_thread is not None)

# frames should advance only when you step:
w = m.scenario.world
f0 = w.get_snapshot().frame
# run one manual step:
_ = my_run_scenario_step(leaderboard_evaluator, "Started", "", n_steps=3)
f1 = w.get_snapshot().frame
print("advanced by manual step:", f1 > f0)


In [ ]:
leaderboard_evaluator.manager._running = True

In [ ]:
w = leaderboard_evaluator.manager.scenario.world
f0 = w.get_snapshot().frame
print("f0", f0)
w.tick()
f1 = w.get_snapshot().frame
print("f1", f1)
print("Frame advanced:", f1 > f0)

print("running:", leaderboard_evaluator.manager._running)
print("scenario:", leaderboard_evaluator.manager.scenario)


In [ ]:
print("running:", leaderboard_evaluator.manager._running)
print("scenario:", leaderboard_evaluator.manager.scenario)


### Move with object without pause the running scenarios

In [ ]:
# vehicles_state[0]['x']=560
# vehicles_state[0]['y']=3910.55517578125

In [ ]:
# scenario_vehicle_state

In [ ]:
### Restore state 
# 1) restore other vehicles 

# def restore_vehicles(world_obj, vehicles_state_json):
#     actors = world_obj.get_actors()
#     vehicles = {v.id: v for v in actors.filter("vehicle.*")}  # map id → actor

#     for rec in vehicles_state_json[::-1]:
#         vid = rec["id"]
#         if vid in vehicles:
#             v = vehicles[vid]
#             if "pitch" in rec:    
#                 set_carla_loc_rot(v, rec["x"], rec["y"], rec["z"], rec["yaw"], rec["pitch"], rec["roll"])
#                 print(f"Moved vehicle {vid} ({v.type_id}) to {rec['x']}, {rec['y']}, {rec['z']}")
#             else:
#                 set_carla_loc_rot(v, rec["x"], rec["y"], rec["z"])
#                 print(f"Vehicle id {vid} not found in world (maybe needs respawn).")
#         else:
#             print(f"Vehicle id {vid} not found in world (maybe needs respawn).")

# restore_vehicles(world_obj, vehicles_state)

# 2) restore ego vehicle 



# set_carla_loc_rot(carla_vehicle_object=leaderboard_evaluator.manager.scenario.list_scenarios[0].other_actors[0], x=568.29, y=3914, z=371.2144, yaw=179.72, pitch=-0.458, roll=0)
# 4) restore scenario manager 



# 5) restore watchdog 




In [ ]:


restore_scenario_vehicles(leaderboard_evaluator.manager.scenario.list_scenarios[0], scenario_vehicle_state)

In [ ]:
leaderboard_evaluator.manager.scenario.list_scenarios[0].other_actors

In [ ]:
leaderboard_evaluator.manager.scenario.list_scenarios[0].other_actors[0].id

In [ ]:
# Grab CARLA actors from your scenario
actors = leaderboard_evaluator.manager.scenario.list_scenarios[0].other_actors   # list of carla.Vehicle

# Build a lookup by ID
id_to_actor = {a.id: a for a in actors}
id_to_actor

In [ ]:
leaderboard_evaluator.manager.scenario.list_scenarios[0].other_actors[0

In [ ]:
scenario_vehicle_state

In [ ]:
leaderboard_evaluator.manager.scenario.list_scenarios[0].other_actors[0].get_location().x
set_carla_loc_rot(carla_vehicle_object=leaderboard_evaluator.manager.scenario.list_scenarios[0].other_actors[1], x=568.29, y=3914, z=371.2144, yaw=179.72, pitch=-0.458, roll=0)



In [ ]:

set_carla_loc_rot(carla_vehicle_object=leaderboard_evaluator.manager.scenario.list_scenarios[0].other_actors[0], x=568.29, y=3914, z=371.2144, yaw=179.72, pitch=-0.458, roll=0)

set_carla_loc_rot(carla_vehicle_object=leaderboard_evaluator.manager.ego_vehicles[0], x=578.208984375, y=3910, z=371.29, yaw=179.76, pitch=0.708, roll=0)

In [ ]:
### The method to move object (to snapshot)
### move bike closer

# set_carla_loc_rot(carla_vehicle_object=leaderboard_evaluator.manager.scenario.list_scenarios[0].other_actors[0], x=568.29, y=3914, z=371.2144, yaw=179.72, pitch=-0.458, roll=0)

In [ ]:
## relocate my car

# set_carla_loc_rot(carla_vehicle_object=leaderboard_evaluator.manager.ego_vehicles[0], x=578.208984375, y=3910, z=371.29, yaw=179.76, pitch=0.708, roll=0)

In [ ]:
import carla, threading, time

# ---------- STOP / START MANAGER ----------

def stop_manager(manager):
    manager._running = False

    # stop builder thread
    t = getattr(manager, "_scenario_thread", None)
    if t and t.is_alive() and threading.current_thread() is not t:
        t.join(timeout=2.0)
    manager._scenario_thread = None

    # stop tick thread if we created one
    tt = getattr(manager, "_tick_thread", None)
    if tt and tt.is_alive() and threading.current_thread() is not tt:
        # give it a moment to exit its loop
        tt.join(timeout=2.0)
    manager._tick_thread = None

def start_manager(manager):
    if not getattr(manager, "_running", False):
        manager._running = True

    # (A) builder thread
    if not getattr(manager, "_scenario_thread", None) or not manager._scenario_thread.is_alive():
        manager._scenario_thread = threading.Thread(
            target=manager.build_scenarios_loop,
            args=(manager._debug_mode > 0,),
            daemon=True
        )
        manager._scenario_thread.start()

    if manager._running:
        manager._tick_scenario()

    # (B) tick loop
    # tt = getattr(manager, "_tick_thread", None)
    # if tt is None or not (hasattr(tt, "is_alive") and tt.is_alive()):
    #     def _tick_loop():
    #         while manager._running:
    #             try:
    #                 manager._tick_scenario()
    #             except Exception as e:
    #                 print("[tick_loop] exception:", e)
    #                 time.sleep(0.01)
    #     manager._tick_thread = threading.Thread(target=_tick_loop, daemon=True)
    #     manager._tick_thread.start()


# ---------- PAUSE → RESTORE → RESUME (0.9.15) ----------

def pause_restore_resume(client, world, manager, vehicles_state):
    """
    client : carla.Client  (e.g., leaderboard_evaluator.client)
    world  : carla.World   (e.g., leaderboard_evaluator.manager.scenario.world)
    """

    # 1) Pause manager/threads
    stop_manager(manager)

    # 2) Put world into sync (pause) and remember previous settings
    prev = world.get_settings()
    if not prev.synchronous_mode:
        new = carla.WorldSettings()
        new.no_rendering_mode   = prev.no_rendering_mode
        new.synchronous_mode    = True
        new.fixed_delta_seconds = prev.fixed_delta_seconds or 0.05
        world.apply_settings(new)

    # 3) Build atomic batch (teleport + zero velocities)
    batch = []
    for rec in vehicles_state:
        vid = int(rec["id"])
        tf  = carla.Transform(
            carla.Location(float(rec["x"]), float(rec["y"]), float(rec["z"])),
            carla.Rotation(
                yaw=float(rec.get("yaw",   0.0)),
                pitch=float(rec.get("pitch", 0.0)),
                roll=float(rec.get("roll",  0.0))
            )
        )
        batch += [
            carla.command.SetSimulatePhysics(vid, False),
            carla.command.ApplyTransform(vid, tf),
            carla.command.ApplyTargetVelocity(vid, carla.Vector3D(0,0,0)),
            carla.command.ApplyTargetAngularVelocity(vid, carla.Vector3D(0,0,0)),
            carla.command.SetSimulatePhysics(vid, True),
        ]

    # 4) Apply atomically and advance a couple frames
    client.apply_batch_sync(batch, True)   # True == do_tick in 0.9.15
    world.tick()
    world.tick()

    # 5) Restore previous world settings if we changed them
    if not prev.synchronous_mode:
        world.apply_settings(prev)

    # 6) Resume manager (builder + tick)
    start_manager(manager)


In [ ]:
pause_restore_resume(
    leaderboard_evaluator.client,
    leaderboard_evaluator.manager.scenario.world,
    leaderboard_evaluator.manager,
    vehicles_state,   # your saved list of dicts
)

In [ ]:
leaderboard_evaluator.manager._tick

In [ ]:
world_obj = leaderboard_evaluator.manager.scenario.world

In [ ]:
world_obj

In [ ]:
def stop_manager(manager):
    # tell all loops to stop
    manager._running = False

    # stop builder thread if present
    t = getattr(manager, "_scenario_thread", None)
    if t and t.is_alive():
        if threading.current_thread() is t:
            # don't join yourself
            pass
        else:
            t.join(timeout=2.0)
    manager._scenario_thread = None

def start_manager(manager):
    if not getattr(manager, "_running", False):
        manager._running = True

    # (A) builder thread (spawns/arms scenarios)
    if not getattr(manager, "_scenario_thread", None) or not manager._scenario_thread.is_alive():
        manager._scenario_thread = threading.Thread(
            target=manager.build_scenarios_loop,
            args=(manager._debug_mode > 0,),
            daemon=True
        )
        manager._scenario_thread.start()

    # (B) tick loop (actually advances the scenario every frame)
    # if your codebase already has a dedicated tick thread, use that instead.
    if not hasattr(manager, "_tick_thread") or not manager._tick_thread.is_alive():
        def _tick_loop():
            # guard against exceptions so the thread doesn’t die silently
            while manager._running:
                try:
                    manager._tick_scenario()
                except Exception as e:
                    # log and continue; adjust to your logger if needed
                    print("[tick_loop] exception:", e)
                    time.sleep(0.01)
        manager._tick_thread = threading.Thread(target=_tick_loop, daemon=True)
        manager._tick_thread.start()

# stop_manager(leaderboard_evaluator.manager)

def pause_restore_resume(world, manager, vehicles_state):
    # ---- Pause all activity ----
    stop_manager(manager)

    # switch to sync so the world stops advancing until we tick
    settings = world.get_settings()
    prev = carla.WorldSettings(
        no_rendering_mode=settings.no_rendering_mode,
        synchronous_mode=settings.synchronous_mode,
        fixed_delta_seconds=settings.fixed_delta_seconds
    )
    if not settings.synchronous_mode:
        settings.synchronous_mode = True
        world.apply_settings(settings)

    # ---- Restore actors atomically ----
    client = world.get_client()
    batch = []
    for rec in vehicles_state:
        tf = carla.Transform(
            carla.Location(float(rec["x"]), float(rec["y"]), float(rec["z"])),
            carla.Rotation(
                yaw=float(rec.get("yaw", 0.0)),
                pitch=float(rec.get("pitch", 0.0)),
                roll=float(rec.get("roll", 0.0))
            )
        )
        vid = int(rec["id"])
        batch.append(carla.command.ApplyTransform(vid, tf))
        batch.append(carla.command.ApplyTargetVelocity(vid, carla.Vector3D(0,0,0)))
        batch.append(carla.command.ApplyTargetAngularVelocity(vid, carla.Vector3D(0,0,0)))

    # apply & tick once; extra ticks help sensors settle
    client.apply_batch_sync(batch, do_tick=True)
    world.tick(); world.tick()

    # ---- Resume previous world mode ----
    if not prev.synchronous_mode:
        settings.synchronous_mode = False
        settings.fixed_delta_seconds = prev.fixed_delta_seconds
        world.apply_settings(settings)

    # ---- Restart manager (builder + tick) ----
    start_manager(manager)

pause_restore_resume(
    leaderboard_evaluator.manager.scenario.world,
    leaderboard_evaluator.manager,
    vehicles_state
)

In [ ]:
world_obj.get_client()

In [ ]:
leaderboard_evaluator.client

In [ ]:
### stop scenario manager

# print("leaderboard_evaluator.manager._scenario_thread.__dict__._is_stopped", leaderboard_evaluator.manager._scenario_thread._is_stopped)
print("leaderboard_evaluator.manager._running", leaderboard_evaluator.manager._running)
leaderboard_evaluator.manager._running = False
try:
    leaderboard_evaluator.manager._scenario_thread.join()
except:
    pass
leaderboard_evaluator.manager._scenario_thread = None
# print("leaderboard_evaluator.manager._scenario_thread.__dict__._is_stopped", leaderboard_evaluator.manager._scenario_thread._is_stopped)
print("leaderboard_evaluator.manager._running", leaderboard_evaluator.manager._running)

In [ ]:
### Start the scenario manager 
def start_builder_thread(manager):
    if getattr(manager, "_scenario_thread", None) and manager._scenario_thread.is_alive():
        return  # already running
    manager._running = True
    manager._scenario_thread = threading.Thread(
        target=manager.build_scenarios_loop,
        args=(manager._debug_mode > 0,),
        daemon=True
    )
    manager._scenario_thread.start()

    if manager._running:
        manager._tick_scenario()

# start_previous_loop_thread(leaderboard_evaluator.manager)
start_builder_thread(leaderboard_evaluator.manager)


In [ ]:
leaderboard_evaluator.manager._tick_scenario()

In [ ]:
### start scenario manager

# leaderboard_evaluator.manager._tick_scenario()
leaderboard_evaluator.manager._running = True
# CarlaDataProvider.on_carla_tick()


In [ ]:
### continue the previous thread

# def start_previous_loop_thread(leaderboard_evaluator_manager_obj):
#     # Thread for build_scenarios
#     leaderboard_evaluator_manager_obj._running = True
#     leaderboard_evaluator_manager_obj._scenario_thread = threading.Thread(target=leaderboard_evaluator_manager_obj.build_scenarios_loop, args=(leaderboard_evaluator_manager_obj._debug_mode > 0, ))
#     leaderboard_evaluator_manager_obj._scenario_thread.start()
#     print("leaderboard_evaluator.manager._scenario_thread.is_alive()", leaderboard_evaluator.manager._scenario_thread.is_alive())

#     if leaderboard_evaluator_manager_obj._running:
#         leaderboard_evaluator_manager_obj._tick_scenario()

def start_builder_thread(manager):
    if getattr(manager, "_scenario_thread", None) and manager._scenario_thread.is_alive():
        return  # already running
    manager._running = True
    manager._scenario_thread = threading.Thread(
        target=manager.build_scenarios_loop,
        args=(manager._debug_mode > 0,),
        daemon=True
    )
    manager._scenario_thread.start()

    if manager._running:
        manager._tick_scenario()

# start_previous_loop_thread(leaderboard_evaluator.manager)
start_builder_thread(leaderboard_evaluator.manager)


In [ ]:
leaderboard_evaluator.manager._running

In [ ]:
leaderboard_evaluator.manager._scenario_thread.__dict__

In [ ]:
leaderboard_evaluator.manager._scenario_thread

In [ ]:
leaderboard_evaluator.manager.ego_vehicles[0].get_location().x, leaderboard_evaluator.manager.ego_vehicles[0].get_location().y, leaderboard_evaluator.manager.ego_vehicles[0].get_location().z

# leaderboard_evaluator.manager.ego_vehicles[0].attributes

In [ ]:
from carla import Location
new_loc = Location(x=589.208984375, y=3910.0215, z=371.29)

leaderboard_evaluator.manager.ego_vehicles[0].set_location(new_loc)

In [ ]:
leaderboard_evaluator

In [ ]:
leaderboard_evaluator.route_scenario.__class__

In [ ]:
leaderboard_evaluator.manager.scenario.__class__

In [ ]:
leaderboard_evaluator.route_scenario.list_scenarios[0].other_actors[0].attributes
# leaderboard_evaluator.route_scenario.occupied_parking_locations
# leaderboard_evaluator.route_scenario.available_parking_locations
# leaderboard_evaluator.route_scenario._parked_ids
# leaderboard_evaluator.route_scenario.ego_data

In [ ]:
leaderboard_evaluator.route_scenario.list_scenarios[0].other_actors[1].get_location().x

In [ ]:
leaderboard_evaluator.manager.scenario.list_scenarios[0].other_actors[0].get_location().x, leaderboard_evaluator.manager.scenario.list_scenarios[0].other_actors[0].get_location().y, leaderboard_evaluator.manager.scenario.list_scenarios[0].other_actors[0].get_location().z

In [ ]:
from carla import Location
new_loc = Location(x=568.2979, y=3914.0215, z=371.2144)

leaderboard_evaluator.manager.scenario.list_scenarios[0].other_actors[0].set_location(new_loc)


In [ ]:
leaderboard_evaluator.route_scenario.other_actors

In [ ]:
leaderboard_evaluator.manager.scenario.other_actors

In [ ]:
leaderboard_evaluator.manager.scenario.ego_vehicles

In [ ]:
leaderboard_evaluator.manager.scenario.build_scenarios(leaderboard_evaluator.manager.scenario.ego_vehicles)

In [ ]:
leaderboard_evaluator.manager.scenario.remove_all_actors()

In [ ]:
leaderboard_evaluator.manager.ego_vehicles[0].get_location().x

In [ ]:
leaderboard_evaluator.manager

In [ ]:
leaderboard_evaluator.manager.build_scenarios_loop(1)

In [ ]:
leaderboard_evaluator.agent_instance

In [ ]:
if hasattr(leaderboard_evaluator, 'world'):
    world = leaderboard_evaluator.world
    vehicles = world.get_actors().filter('vehicle.*')

vehicles

In [ ]:
# for v in vehicles:
#     print("v", v.attributes)


In [ ]:
leaderboard_evaluator.manager.scenario.list_scenarios
leaderboard_evaluator.manager.scenario.occupied_parking_locations
leaderboard_evaluator.manager.scenario.available_parking_locations
leaderboard_evaluator.manager.scenario._parked_ids
leaderboard_evaluator.manager.scenario.ego_data
leaderboard_evaluator.manager.scenario.behavior_node
leaderboard_evaluator.manager.scenario.other_actors








In [ ]:
leaderboard_evaluator.manager.scenario.list_scenarios[0].remove_all_actors()

In [ ]:
leaderboard_evaluator.route_scenario.other_actors

In [ ]:
# Import the necessary modules for snapshot functionality
import sys
import pickle
from pathlib import Path
from datetime import datetime
import copy

# Add the parent directory to path to import from carla_server
carla_server_path = '/mnt3/Documents/AD_Framework/bench2drive-gymnasium/bench2drive_microservices'
if carla_server_path not in sys.path:
    sys.path.insert(0, carla_server_path)

# Import the WorldSnapshot class
from world_snapshot import WorldSnapshot

# Create a simple simulation state object to hold our snapshot
class SimState:
    def __init__(self):
        self.leaderboard_evaluator = None
        self.snapshots = {}
        self.snapshot_dir = Path("/mnt3/Documents/AD_Framework/bench2drive-gymnasium/bench2drive_microservices/notebooks/debug_snapshots")
        self.snapshot_dir.mkdir(parents=True, exist_ok=True)
        self.step_count = 0
        self.server_id = "notebook_test"
        self.last_observation = None

# Initialize simulation state
sim_state = SimState()
sim_state.leaderboard_evaluator = leaderboard_evaluator
sim_state.step_count = total_steps_run

print(f"Initialized sim_state with leaderboard_evaluator")
print(f"Snapshot directory: {sim_state.snapshot_dir}")
print(f"Current step count: {sim_state.step_count}")

In [ ]:
# Function to save a snapshot (adapted from carla_server.py)
def save_snapshot(snapshot_id=None):
    """Save current world state"""
    try:
        # Verify scenario manager exists
        if not hasattr(sim_state.leaderboard_evaluator, 'manager') or sim_state.leaderboard_evaluator.manager is None:
            raise ValueError("Scenario manager not initialized - cannot snapshot without manager")
        
        # Generate snapshot ID if not provided
        if snapshot_id is None:
            snapshot_id = f"snap_{sim_state.server_id}_{len(sim_state.snapshots)}_{datetime.now().strftime('%H%M%S')}"
        
        # Get world from leaderboard_evaluator
        world = None
        if hasattr(sim_state.leaderboard_evaluator, 'world'):
            world = sim_state.leaderboard_evaluator.world
            print(f"DEBUG: Got world from leaderboard_evaluator: {world}")
        else:
            print("ERROR: leaderboard_evaluator has no world attribute!")
        
        # Debug: Print available attributes
        print("DEBUG: leaderboard_evaluator attributes:")
        print(f"  - Has manager: {hasattr(sim_state.leaderboard_evaluator, 'manager')}")
        print(f"  - Has world: {hasattr(sim_state.leaderboard_evaluator, 'world')}")
        print(f"  - World object: {world}")
        print(f"  - Has agent_instance: {hasattr(sim_state.leaderboard_evaluator, 'agent_instance')}")
        print(f"  - Has route_scenario: {hasattr(sim_state.leaderboard_evaluator, 'route_scenario')}")
        
        # Get current observation if available
        current_observation = {}
        if hasattr(sim_state.leaderboard_evaluator.agent_instance, 'last_input_data'):
            input_data = sim_state.leaderboard_evaluator.agent_instance.last_input_data
            current_observation = {
                'images': {},
                'sensors': {}
            }
            # Store sensor data
            for key, val in input_data.items():
                if key in ['Center', 'Left', 'Right']:
                    current_observation['images'][key] = val[1]  # Store the image array
                else:
                    current_observation['sensors'][key] = val
            print(f"Captured observation with {len(current_observation['images'])} images")
        
        # Capture the snapshot
        phase_marker = f"Step {sim_state.step_count} - {snapshot_id}"
        snapshot = WorldSnapshot.capture(sim_state, world, phase_marker=phase_marker)
        snapshot.snapshot_id = snapshot_id
        
        # Store observation in snapshot
        snapshot.observation = current_observation
        
        # Store a deep copy of the snapshot
        sim_state.snapshots[snapshot_id] = copy.deepcopy(snapshot)
        
        # Save snapshot to disk
        snapshot_file = sim_state.snapshot_dir / f"{snapshot_id}.pkl"
        try:
            with open(snapshot_file, 'wb') as f:
                pickle.dump(snapshot, f)
            print(f"Snapshot saved to disk: {snapshot_file}")
        except Exception as e:
            print(f"Failed to save snapshot to disk: {e}")
        
        # Print snapshot details
        print(f"\\nSnapshot captured: {snapshot_id}")
        print(f"  - Vehicles: {len(snapshot.vehicles)}")
        print(f"  - Has scenario_manager state: {snapshot.scenario_manager is not None}")
        print(f"  - Saved to: {snapshot_file}")
        
        # Log ego vehicle position
        for vehicle_id, vehicle_state in snapshot.vehicles.items():
            if vehicle_state.is_hero:
                print(f"  - Ego position: x={vehicle_state.location['x']:.2f}, y={vehicle_state.location['y']:.2f}, z={vehicle_state.location['z']:.2f}")
                break
        
        if snapshot.scenario_manager:
            print(f"  - Manager tick: {snapshot.scenario_manager.tick_count}")
            print(f"  - Manager running: {snapshot.scenario_manager.running}")
            print(f"  - Ego vehicles: {len(snapshot.scenario_manager.ego_vehicle_ids)}")
        
        return snapshot_id
        
    except Exception as e:
        print(f"Error saving snapshot: {e}")
        import traceback
        traceback.print_exc()
        return None

# Save a snapshot of the current state
print("\\n=== Saving Snapshot of Current State ===")
snapshot_id = save_snapshot("test_snapshot_after_50_steps")

if snapshot_id:
    print(f"\\n✅ Successfully saved snapshot: {snapshot_id}")
    print(f"Total snapshots in memory: {len(sim_state.snapshots)}")
else:
    print("\\n❌ Failed to save snapshot")

In [ ]:
### Run for another 50 steps.

# Run another 50 simulation steps with image saving
print("\n" + "="*60)
print("Running Another 50 Steps with Image Capture")
print("="*60)

# Create new directory for this batch of images
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
save_dir_batch2 = Path(f"/mnt3/Documents/AD_Framework/bench2drive-gymnasium/bench2drive_microservices/notebooks/debug_snapshots/debug_images_batch2_{timestamp}")
save_dir_batch2.mkdir(parents=True, exist_ok=True)
print(f"Images will be saved to: {save_dir_batch2}")

run_step_n = 50  # Run 50 more steps
initial_step_count = total_steps_run

if not crash_flag:
    print(f"\n=== Running {run_step_n} simulation steps with image capture ===")
    print(f"Starting from step {total_steps_run}")
    
    for step_i in range(run_step_n):
        # Run one step
        crash_flag_step, entry_status, crash_message = my_run_scenario_step(
            leaderboard_evaluator, entry_status, crash_message, n_steps=1
        )
        
        if crash_flag_step:
            crash_flag = True
            print(f"Crash detected at step {step_i}")
            break
        
        # Save images after each step
        if hasattr(leaderboard_evaluator.agent_instance, 'last_input_data'):
            # Save images every 1 steps to reduce storage
            if step_i % 1 == 0:
                print(f"\nStep {total_steps_run + step_i}:")
                step_dir = save_debug_images(
                    leaderboard_evaluator.agent_instance, 
                    save_dir_batch2, 
                    total_steps_run + step_i
                )
                
                # Also log vehicle position
                if hasattr(leaderboard_evaluator.agent_instance, '_vehicle') and leaderboard_evaluator.agent_instance._vehicle:
                    vehicle = leaderboard_evaluator.agent_instance._vehicle
                    transform = vehicle.get_transform()
                    print(f"  Vehicle position: x={transform.location.x:.2f}, y={transform.location.y:.2f}, z={transform.location.z:.2f}")
        else:
            if step_i % 10 == 0:
                print(f"Step {total_steps_run + step_i}: Running...")
    
    total_steps_run += step_i + 1
    
print(f"\n=== Completed {step_i + 1} additional steps ===")
print(f"Total steps run in session: {total_steps_run}")
print(f"crash_flag: {crash_flag}")
print(f"entry_status: {entry_status}")
print(f"crash_message: {crash_message}")
print(f"Images saved to: {save_dir_batch2}")

# Display summary
if save_dir_batch2.exists():
    step_dirs = sorted(save_dir_batch2.glob("step_*"))
    print(f"\nSaved {len(step_dirs)} sets of images")
    for step_dir in step_dirs[:3]:  # Show first 3 steps
        images = list(step_dir.glob("*.jpg"))
        print(f"  {step_dir.name}: {len(images)} images")

# Save a snapshot after these 50 steps
print("\n" + "="*60)
print("Saving Snapshot After Additional 50 Steps")
print("="*60)

snapshot_after_100 = save_snapshot(f"snapshot_after_{total_steps_run}_steps")
if snapshot_after_100:
    print(f"✅ Saved snapshot: {snapshot_after_100}")
    print(f"Total snapshots available: {len(sim_state.snapshots)}")
    
    # List all snapshots
    print("\nAll available snapshots:")
    for snap_id in sim_state.snapshots.keys():
        print(f"  - {snap_id}")

# Update sim_state step count
sim_state.step_count = total_steps_run

In [ ]:
### Restore snapshot 

# Restore to a previous snapshot and verify the state
print("\n" + "="*60)
print("Restoring to Previous Snapshot")
print("="*60)

# Display all available snapshots
print("\nAvailable snapshots to restore from:")
for i, snap_id in enumerate(sim_state.snapshots.keys()):
    snapshot = sim_state.snapshots[snap_id]
    print(f"  {i+1}. {snap_id}")
    print(f"     - Vehicles: {len(snapshot.vehicles)}")
    if hasattr(snapshot, 'scenario_manager') and snapshot.scenario_manager:
        print(f"     - Manager tick: {snapshot.scenario_manager.tick_count}")
    # Show ego position in snapshot  
    for vehicle_id, vehicle_state in snapshot.vehicles.items():
        if vehicle_state.is_hero:
            print(f"     - Ego position: x={vehicle_state.location['x']:.2f}, y={vehicle_state.location['y']:.2f}")
            break

# Choose which snapshot to restore (you can change this)
snapshot_to_restore = "test_snapshot_after_50_steps"  # Change this to any snapshot ID you want

print(f"\n>>> Restoring to: {snapshot_to_restore}")

# Get current state BEFORE restore
print("\n=== Current State Before Restore ===")
current_position = None
current_tick_count = None
if hasattr(leaderboard_evaluator.agent_instance, '_vehicle') and leaderboard_evaluator.agent_instance._vehicle:
    vehicle = leaderboard_evaluator.agent_instance._vehicle
    transform = vehicle.get_transform()
    current_position = (transform.location.x, transform.location.y, transform.location.z)
    print(f"Current position: x={current_position[0]:.2f}, y={current_position[1]:.2f}, z={current_position[2]:.2f}")

if hasattr(leaderboard_evaluator, 'manager') and leaderboard_evaluator.manager:
    current_tick_count = leaderboard_evaluator.manager.tick_count
    print(f"Current manager tick count: {current_tick_count}")
    print(f"Current total_steps_run variable: {total_steps_run}")

# Save images BEFORE restore for comparison
print("\n=== Saving Images Before Restore ===")
timestamp_before = datetime.now().strftime("%Y%m%d_%H%M%S")
before_restore_dir = sim_state.snapshot_dir / f"before_restore_{timestamp_before}"
before_restore_dir.mkdir(parents=True, exist_ok=True)

if hasattr(leaderboard_evaluator.agent_instance, 'last_input_data'):
    save_debug_images(leaderboard_evaluator.agent_instance, before_restore_dir, current_tick_count if current_tick_count else total_steps_run)
    print(f"Saved 'before' images to: {before_restore_dir}")

# Perform the restore
try:
    print(f"\n=== Starting Restore of Snapshot: {snapshot_to_restore} ===")
    
    # Get the snapshot
    if snapshot_to_restore not in sim_state.snapshots:
        # Try to load from disk if not in memory
        snapshot_file = sim_state.snapshot_dir / f"{snapshot_to_restore}.pkl"
        if snapshot_file.exists():
            print(f"Loading snapshot from disk: {snapshot_file}")
            with open(snapshot_file, 'rb') as f:
                snapshot = pickle.load(f)
                sim_state.snapshots[snapshot_to_restore] = snapshot
        else:
            raise ValueError(f"Snapshot not found: {snapshot_to_restore}")
    
    snapshot = sim_state.snapshots[snapshot_to_restore]
    print(f"Found snapshot with {len(snapshot.vehicles)} vehicles")
    
    # Get the world
    world = sim_state.leaderboard_evaluator.world
    if world is None:
        raise ValueError("World is not available")
    
    # CRITICAL FIX: The restore method signature is def restore(self, sim_state, world)
    print("\nRestoring world state...")
    print(f"DEBUG: Calling restore with sim_state={sim_state}, world={world}")
    
    # Call restore with correct parameter order
    snapshot.restore(sim_state, world)
    
    # CRITICAL: Force the manager tick count to be restored
    if hasattr(snapshot, 'scenario_manager') and snapshot.scenario_manager:
        if hasattr(leaderboard_evaluator, 'manager') and leaderboard_evaluator.manager:
            restored_tick = snapshot.scenario_manager.tick_count
            leaderboard_evaluator.manager.tick_count = restored_tick
            print(f"\nForced manager tick count to: {restored_tick}")
            
            # Also update our tracking variables to match
            total_steps_run = restored_tick
            sim_state.step_count = restored_tick
            print(f"Updated total_steps_run to: {total_steps_run}")
    
    # CRITICAL FIX: Reinitialize watchdogs IMMEDIATELY after restore
    print("\n=== Fixing Watchdog Issues After Restore ===")
    from srunner.scenariomanager.watchdog import Watchdog
    
    if hasattr(leaderboard_evaluator, 'manager') and leaderboard_evaluator.manager:
        # Fix manager watchdog
        if not hasattr(leaderboard_evaluator.manager, '_watchdog') or leaderboard_evaluator.manager._watchdog is None:
            print("Reinitializing manager watchdog...")
            leaderboard_evaluator.manager._watchdog = Watchdog(leaderboard_evaluator.manager._timeout)
            leaderboard_evaluator.manager._watchdog.start()
            print("✅ Manager watchdog reinitialized")
        
        # Fix agent watchdog
        if not hasattr(leaderboard_evaluator.manager, '_agent_watchdog') or leaderboard_evaluator.manager._agent_watchdog is None:
            print("Reinitializing agent watchdog...")
            leaderboard_evaluator.manager._agent_watchdog = Watchdog(leaderboard_evaluator.manager._timeout)
            leaderboard_evaluator.manager._agent_watchdog.start()
            print("✅ Agent watchdog reinitialized")
    
    # Force a world tick to apply restored positions
    print("\nForcing world tick to apply restored positions...")
    if hasattr(leaderboard_evaluator, 'world') and leaderboard_evaluator.world:
        try:
            leaderboard_evaluator.world.tick()
            print("✅ World ticked successfully")
        except Exception as e:
            print(f"⚠️ Failed to tick world: {e}")
    
    # Verify ego position after restore
    print("\n=== State After Restore ===")
    if hasattr(leaderboard_evaluator.agent_instance, '_vehicle') and leaderboard_evaluator.agent_instance._vehicle:
        vehicle = leaderboard_evaluator.agent_instance._vehicle
        transform = vehicle.get_transform()
        restored_position = (transform.location.x, transform.location.y, transform.location.z)
        print(f"Position AFTER restore: x={restored_position[0]:.2f}, y={restored_position[1]:.2f}, z={restored_position[2]:.2f}")
        
        # Calculate distance moved
        if current_position:
            distance = ((restored_position[0] - current_position[0])**2 + 
                       (restored_position[1] - current_position[1])**2 + 
                       (restored_position[2] - current_position[2])**2)**0.5
            print(f"Vehicle moved {distance:.2f} units during restore")
    
    if hasattr(leaderboard_evaluator, 'manager') and leaderboard_evaluator.manager:
        print(f"Manager tick count after restore: {leaderboard_evaluator.manager.tick_count}")
    
    # Restore observation if available and save restored images
    if hasattr(snapshot, 'observation') and snapshot.observation:
        sim_state.last_observation = snapshot.observation
        print(f"\nRestored observation with {len(snapshot.observation.get('images', {}))} images")
        
        # Save the restored images
        print("\n=== Saving Restored Images ===")
        timestamp_after = datetime.now().strftime("%Y%m%d_%H%M%S")
        after_restore_dir = sim_state.snapshot_dir / f"after_restore_{timestamp_after}"
        after_restore_dir.mkdir(parents=True, exist_ok=True)
        
        # Save each camera image from the restored observation
        for camera_id in ['Center', 'Left', 'Right']:
            if camera_id in snapshot.observation.get('images', {}):
                img_array = snapshot.observation['images'][camera_id]
                if len(img_array.shape) == 3:  # Make sure it's an image
                    # Convert and save
                    bgr_image = cv2.cvtColor(img_array[:, :, :3], cv2.COLOR_RGB2BGR)
                    image_path = after_restore_dir / f"restored_{camera_id}.jpg"
                    cv2.imwrite(str(image_path), bgr_image)
                    print(f"  Saved restored {camera_id} image to {image_path.name}")
        
        print(f"Restored images saved to: {after_restore_dir}")
    
    print(f"\n✅ Successfully restored snapshot: {snapshot_to_restore}")
    print(f"   Simulation state has been reset to tick {total_steps_run}")
    
except Exception as e:
    print(f"\n❌ Error restoring snapshot: {e}")
    import traceback
    traceback.print_exc()

# Run 50 steps after restore to fully demonstrate it's working
print("\n" + "="*60)
print("Running 50 Steps After Restore with Image Capture")
print("="*60)

timestamp_verify = datetime.now().strftime("%Y%m%d_%H%M%S")
verify_dir = sim_state.snapshot_dir / f"verify_after_restore_50steps_{timestamp_verify}"
verify_dir.mkdir(parents=True, exist_ok=True)

verification_steps = 50  # Run 50 steps to show proper restoration

# CRITICAL: Use the restored tick count as the starting point
restored_step_start = leaderboard_evaluator.manager.tick_count if hasattr(leaderboard_evaluator, 'manager') else total_steps_run

print(f"Starting from restored tick: {restored_step_start}")
print(f"Will run {verification_steps} steps and save images every step")
print(f"Expected to reach tick: {restored_step_start + verification_steps}\n")

for step_i in range(verification_steps):
    # Run one step
    crash_flag_step, entry_status, crash_message = my_run_scenario_step(
        leaderboard_evaluator, entry_status, crash_message, n_steps=1
    )
    
    if crash_flag_step:
        print(f"Crash detected at verification step {step_i}")
        break
    
    # Get current tick from manager
    current_tick = leaderboard_evaluator.manager.tick_count if hasattr(leaderboard_evaluator, 'manager') else (restored_step_start + step_i)
    
    # Save images every step to show progression
    if hasattr(leaderboard_evaluator.agent_instance, 'last_input_data'):
        print(f"\nTick {current_tick} (Verification step {step_i}):")
        step_dir = save_debug_images(
            leaderboard_evaluator.agent_instance, 
            verify_dir, 
            current_tick  # Use actual tick count, not step index
        )
        
        # Also log vehicle position
        if hasattr(leaderboard_evaluator.agent_instance, '_vehicle') and leaderboard_evaluator.agent_instance._vehicle:
            vehicle = leaderboard_evaluator.agent_instance._vehicle
            transform = vehicle.get_transform()
            print(f"  Vehicle position: x={transform.location.x:.2f}, y={transform.location.y:.2f}, z={transform.location.z:.2f}")
    else:
        # Show progress
        if step_i % 10 == 0:
            print(f"Running tick {current_tick} (Verification step {step_i})...")

# Update total steps based on actual manager state
if hasattr(leaderboard_evaluator, 'manager'):
    total_steps_run = leaderboard_evaluator.manager.tick_count
else:
    total_steps_run = restored_step_start + step_i + 1

# Final position and summary
print(f"\n=== Completed {step_i + 1} verification steps ===")
print(f"Started from tick: {restored_step_start}")
print(f"Ended at tick: {total_steps_run}")
print(f"Total verification steps run: {step_i + 1}")

if hasattr(leaderboard_evaluator.agent_instance, '_vehicle') and leaderboard_evaluator.agent_instance._vehicle:
    vehicle = leaderboard_evaluator.agent_instance._vehicle
    transform = vehicle.get_transform()
    print(f"\nFinal position after {verification_steps} steps: x={transform.location.x:.2f}, y={transform.location.y:.2f}, z={transform.location.z:.2f}")
    
    # Compare with original snapshot position
    for vehicle_id, vehicle_state in snapshot.vehicles.items():
        if vehicle_state.is_hero:
            original_x = vehicle_state.location['x']
            original_y = vehicle_state.location['y']
            distance_traveled = ((transform.location.x - original_x)**2 + 
                               (transform.location.y - original_y)**2)**0.5
            print(f"Distance traveled from restored point: {distance_traveled:.2f} units")
            break

print(f"\n✅ Successfully ran {step_i + 1} steps after restore!")
print(f"Verification images saved to: {verify_dir}")

# Count saved images and check their names
if verify_dir.exists():
    step_dirs = sorted(verify_dir.glob("step_*"))
    print(f"Saved {len(step_dirs)} sets of images")
    print("Image directories created:")
    for step_dir in step_dirs[:5]:  # Show first 5
        print(f"  - {step_dir.name}")
    if len(step_dirs) > 5:
        print(f"  ... and {len(step_dirs) - 5} more")

# Summary of all saved images
print("\n" + "="*60)
print("RESTORE VERIFICATION SUMMARY")
print("="*60)
print(f"1. Before restore images: {before_restore_dir}")
print(f"2. Restored snapshot images: {after_restore_dir if 'after_restore_dir' in locals() else 'Not saved (no observation in snapshot)'}")
print(f"3. After restore verification (50 steps): {verify_dir}")
print(f"\nThe simulation was restored to tick {restored_step_start} and ran to tick {total_steps_run}")
print(f"This confirms the restore {'WORKED - properly restored to step 50!' if restored_step_start == 50 else f'FAILED - started from tick {restored_step_start} instead of 50'}")

# CRITICAL: Check if the first image directory shows step 50 or 100
if verify_dir.exists():
    step_dirs = sorted(verify_dir.glob("step_*"))
    if step_dirs:
        first_dir = step_dirs[0].name
        print(f"\n🔍 CRITICAL CHECK - First verification image directory: {first_dir}")
        if "step_0050" in first_dir or "step_0051" in first_dir:
            print("✅ Images show restore WORKED - continuing from step 50!")
        elif "step_0100" in first_dir or "step_0101" in first_dir:
            print("❌ Images show restore FAILED - still at step 100!")
        else:
            print(f"⚠️ Unexpected step number in first directory: {first_dir}")
            
print("\n⚠️ IMPORTANT: Check the images in the verification directory to confirm:")
print(f"   {verify_dir}")
print("   The images should show the scenery from step 50, NOT step 100!")

# Optional: Save a new snapshot after these 50 verification steps
print("\n" + "="*60)
print("Saving New Snapshot After Verification")
print("="*60)
sim_state.step_count = total_steps_run
new_snapshot_id = save_snapshot(f"snapshot_after_restore_verification_{total_steps_run}_steps")
if new_snapshot_id:
    print(f"✅ Saved new snapshot: {new_snapshot_id}")
    print(f"Total snapshots available: {len(sim_state.snapshots)}")

In [ ]:
### Fix the restore by reinitializing watchdog and forcing tick

# After restore, we need to fix the watchdog issue and force a proper world tick
print("\n" + "="*60)
print("Fixing Restore Issues")
print("="*60)

# Fix 1: Reinitialize the watchdog that was set to None during restore
if hasattr(leaderboard_evaluator, 'manager') and leaderboard_evaluator.manager:
    from srunner.scenariomanager.watchdog import Watchdog
    
    # Recreate the watchdog if it's None
    if not hasattr(leaderboard_evaluator.manager, '_watchdog') or leaderboard_evaluator.manager._watchdog is None:
        print("Reinitializing manager watchdog...")
        leaderboard_evaluator.manager._watchdog = Watchdog(leaderboard_evaluator.manager._timeout)
        leaderboard_evaluator.manager._watchdog.start()
        print("✅ Watchdog reinitialized")
    
    # Also fix agent watchdog if needed
    if not hasattr(leaderboard_evaluator.manager, '_agent_watchdog') or leaderboard_evaluator.manager._agent_watchdog is None:
        print("Reinitializing agent watchdog...")
        leaderboard_evaluator.manager._agent_watchdog = Watchdog(leaderboard_evaluator.manager._timeout)
        leaderboard_evaluator.manager._agent_watchdog.start()
        print("✅ Agent watchdog reinitialized")

# Fix 2: Force a world tick to apply the restored positions
print("\nForcing world tick to apply restored positions...")
if hasattr(leaderboard_evaluator, 'world') and leaderboard_evaluator.world:
    try:
        # Force synchronous tick
        leaderboard_evaluator.world.tick()
        print("✅ World ticked successfully")
    except Exception as e:
        print(f"❌ Failed to tick world: {e}")

# Fix 3: Verify ego vehicle position after tick
if hasattr(leaderboard_evaluator.agent_instance, '_vehicle') and leaderboard_evaluator.agent_instance._vehicle:
    vehicle = leaderboard_evaluator.agent_instance._vehicle
    transform = vehicle.get_transform()
    print(f"\nEgo position after fixes: x={transform.location.x:.2f}, y={transform.location.y:.2f}, z={transform.location.z:.2f}")
    
    # Check if we're at the right position now
    expected_x = 580.47  # From snapshot
    expected_y = 3910.70
    distance_from_expected = ((transform.location.x - expected_x)**2 + (transform.location.y - expected_y)**2)**0.5
    if distance_from_expected < 5.0:  # Within 5 meters
        print(f"✅ Position restored correctly! (within {distance_from_expected:.2f}m of expected)")
    else:
        print(f"⚠️ Position still incorrect! {distance_from_expected:.2f}m from expected position")
        print(f"   Expected: x={expected_x:.2f}, y={expected_y:.2f}")
        print(f"   Got: x={transform.location.x:.2f}, y={transform.location.y:.2f}")

print("\nRestore fixes applied. Ready to continue simulation.")

In [ ]:
### Run verification steps after fixing the restore

print("\n" + "="*60)
print("Running Verification After Restore Fixes")
print("="*60)

timestamp_verify = datetime.now().strftime("%Y%m%d_%H%M%S")
verify_dir = sim_state.snapshot_dir / f"verify_after_fix_{timestamp_verify}"
verify_dir.mkdir(parents=True, exist_ok=True)

verification_steps = 50
restored_step_start = leaderboard_evaluator.manager.tick_count if hasattr(leaderboard_evaluator, 'manager') else 50

print(f"Starting from tick: {restored_step_start}")
print(f"Will run {verification_steps} steps and save images every 5 steps")
print(f"Images will be saved to: {verify_dir}\n")

for step_i in range(verification_steps):
    # Run one step
    crash_flag_step, entry_status, crash_message = my_run_scenario_step(
        leaderboard_evaluator, entry_status, crash_message, n_steps=1
    )
    
    if crash_flag_step:
        print(f"Crash detected at verification step {step_i}")
        break
    
    # Get current tick from manager
    current_tick = leaderboard_evaluator.manager.tick_count if hasattr(leaderboard_evaluator, 'manager') else (restored_step_start + step_i)
    
    # Save images every 5 steps
    if step_i % 5 == 0:
        if hasattr(leaderboard_evaluator.agent_instance, 'last_input_data'):
            print(f"\nTick {current_tick} (Step {step_i}):")
            step_dir = save_debug_images(
                leaderboard_evaluator.agent_instance, 
                verify_dir, 
                current_tick
            )
            
            # Log vehicle position
            if hasattr(leaderboard_evaluator.agent_instance, '_vehicle') and leaderboard_evaluator.agent_instance._vehicle:
                vehicle = leaderboard_evaluator.agent_instance._vehicle
                transform = vehicle.get_transform()
                print(f"  Vehicle at: x={transform.location.x:.2f}, y={transform.location.y:.2f}, z={transform.location.z:.2f}")

# Update total steps
if hasattr(leaderboard_evaluator, 'manager'):
    total_steps_run = leaderboard_evaluator.manager.tick_count
else:
    total_steps_run = restored_step_start + step_i + 1

print(f"\n=== Verification Complete ===")
print(f"Ran {step_i + 1} steps successfully")
print(f"Final tick: {total_steps_run}")
print(f"Images saved to: {verify_dir}")

# Check the saved images
if verify_dir.exists():
    step_dirs = sorted(verify_dir.glob("step_*"))
    print(f"\nSaved {len(step_dirs)} sets of images:")
    for step_dir in step_dirs[:3]:
        print(f"  - {step_dir.name}")
    
    # CRITICAL: Check if the first image directory shows step 50 or 100
    if step_dirs:
        first_dir = step_dirs[0].name
        print(f"\n🔍 First image directory: {first_dir}")
        if "step_0050" in first_dir or "step_0051" in first_dir:
            print("✅ Images show restore WORKED - continuing from step 50!")
        elif "step_0100" in first_dir or "step_0101" in first_dir:
            print("❌ Images show restore FAILED - still at step 100!")
        else:
            print(f"⚠️ Unexpected step number in first directory: {first_dir}")

print("\n" + "="*60)
print("FINAL VERIFICATION")
print("="*60)
print("Check these directories to verify the restore worked:")
print(f"1. {verify_dir}/step_{restored_step_start:04d} - Should show scenery from step 50")
print(f"2. Compare with original step 50 images")
print("\nIf the images match the original step 50 scenery, the restore worked!")
print("If they show step 100 scenery, the restore failed to actually reset the world state.")

In [ ]:

# def set_carla_loc_rot(carla_vehicle_object, x, y, z, yaw=None, pitch=None, roll=None):
#     # Create a new location (x, y, z)
#     print("x,y,z", x,y,z)
#     print("yaw, pitch, roll", yaw, pitch, roll)
#     loc = carla.Location(x=x, y=y, z=z)
#     # Create a new rotation (pitch, yaw, roll)
#     if yaw is None:
#         yaw = carla_vehicle_object.get_transform().rotation.yaw
#     if pitch is None:
#         pitch = carla_vehicle_object.get_transform().rotation.pitch
#     if roll is None:
#         roll = carla_vehicle_object.get_transform().rotation.roll
#     rot = carla.Rotation(pitch=pitch, yaw=yaw, roll=roll)
#     print("loc", loc)
#     print("rot", rot)
#     new_transform = carla.Transform(loc, rot)

#     carla_vehicle_object.set_transform(new_transform)
#     ## Check and return print instead
#     if carla_vehicle_object.get_location().x != x or carla_vehicle_object.get_location().y != y or carla_vehicle_object.get_location().z != z:
#         print("location is not set correctly")

In [ ]:
import carla, threading, time

# ---------- STOP / START (builder-only) ----------

def stop_manager(manager):
    manager._running = False

    t = getattr(manager, "_scenario_thread", None)
    if t and hasattr(t, "is_alive") and t.is_alive() and threading.current_thread() is not t:
        t.join(timeout=2.0)
    manager._scenario_thread = None

    # if a tick thread ever existed, stop it too (safe even if missing)
    tt = getattr(manager, "_tick_thread", None)
    if tt and hasattr(tt, "is_alive") and tt.is_alive() and threading.current_thread() is not tt:
        tt.join(timeout=2.0)
    manager._tick_thread = None

def reset_tree(tree):
    for node in tree.iterate():  # breadth-first
        try:
            node.terminate(py_trees.common.Status.INVALID)
        except Exception:
            pass
        try:
            node.initialise()
        except Exception:
            pass
    tree.status = py_trees.common.Status.RUNNING

def start_manager_builder_only(manager):
    """Start ONLY the builder loop; NO auto-ticking."""
    manager._running = True
    t = getattr(manager, "_scenario_thread", None)
    if t is None or not (hasattr(t, "is_alive") and t.is_alive()):
        manager._scenario_thread = threading.Thread(
            target=manager.build_scenarios_loop,
            args=(manager._debug_mode > 0,),
            daemon=True
        )
        manager._scenario_thread.start()
    # <-- DO NOT call _tick_scenario() here; you will step manually.

def _to_transform(entry):
    if isinstance(entry, tuple) and len(entry) >= 1:
        entry = entry[0]

    if isinstance(entry, carla.Transform):
        return entry

    if hasattr(entry, "transform"):
        return entry.transform

    if isinstance(entry, dict):
        x = float(entry.get("x", 0.0))
        y = float(entry.get("y", 0.0))
        z = float(entry.get("z", 0.0))
        yaw = float(entry.get("yaw", 0.0))
        return carla.Transform(
            carla.Location(x, y, z),
            carla.Rotation(yaw=yaw)
        )
    raise TypeError(f"Don't know how to convert plan entry {type(entry)} to carla.Transform")

def snap_ego_to_route_start(manager):
    ego = manager.ego_vehicles[0]
    scn = manager.scenario

    plan = None
    if hasattr(scn, "_global_plan") and scn._global_plan:
        plan = scn._global_plan
    elif hasattr(scn, "global_plan") and scn.global_plan:
        plan = scn.global_plan
    elif hasattr(scn, "route") and scn.route:
        plan = scn.route

    if plan:
        tf = _to_transform(plan[0])
    else:
        # fallback: nearest driving waypoint from current ego loc
        wmap = manager.scenario.world.get_map()
        wp = wmap.get_waypoint(
            ego.get_location(),
            project_to_road=True,
            lane_type=carla.LaneType.Driving
        )
        tf = wp.transform

    ego.set_transform(tf)
    ego.set_target_velocity(carla.Vector3D(0,0,0))
    ego.set_target_angular_velocity(carla.Vector3D(0,0,0))

# def snap_ego_to_route_start(manager):
#     ego = manager.ego_vehicles[0]
#     scn = manager.scenario

#     wp = None
#     # common attributes across LB/ScenarioRunner variants:
#     if hasattr(scn, "_global_plan") and scn._global_plan:
#         # typically list of (carla.Waypoint, RoadOption)
#         wp = scn._global_plan[0][0]
#     elif hasattr(scn, "global_plan") and scn.global_plan:
#         wp = scn.global_plan[0][0]
#     elif hasattr(scn, "route") and scn.route:
#         # sometimes route is list of waypoints directly
#         wp = scn.route[0]

#     if wp is None:
#         # fallback: use current location's nearest drivable waypoint
#         wmap = manager.scenario.world.get_map()
#         wp = wmap.get_waypoint(ego.get_location(), project_to_road=True, lane_type=carla.LaneType.Driving)


#     print("wp", wp)
#     tf = wp.transform
#     ego.set_transform(tf)
#     ego.set_target_velocity(carla.Vector3D(0,0,0))
#     ego.set_target_angular_velocity(carla.Vector3D(0,0,0))

def pause_restore_resume(client, world, manager, vehicles_state, keep_sync=True):
    stop_manager(manager)

    prev = world.get_settings()
    changed = False
    if not prev.synchronous_mode:
        new = carla.WorldSettings()
        new.no_rendering_mode   = prev.no_rendering_mode
        new.synchronous_mode    = True
        new.fixed_delta_seconds = prev.fixed_delta_seconds or 0.05
        world.apply_settings(new)
        changed = True

    # Build batch for all vehicles (NPCs + ego). If you prefer to exclude ego,
    # filter vehicles_state by id != ego.id instead.
    batch = []
    for rec in vehicles_state:
        vid = int(rec["id"])
        tf  = carla.Transform(
            carla.Location(float(rec["x"]), float(rec["y"]), float(rec["z"])),
            carla.Rotation(
                yaw=float(rec.get("yaw",   0.0)),
                pitch=float(rec.get("pitch", 0.0)),
                roll=float(rec.get("roll",  0.0))
            )
        )
        batch += [
            carla.command.SetSimulatePhysics(vid, False),
            carla.command.ApplyTransform(vid, tf),
            carla.command.ApplyTargetVelocity(vid, carla.Vector3D(0,0,0)),
            carla.command.ApplyTargetAngularVelocity(vid, carla.Vector3D(0,0,0)),
            carla.command.SetSimulatePhysics(vid, True),
        ]

    # Apply batch & settle
    client.apply_batch_sync(batch, True)
    world.tick(); world.tick()

    # NOW snap ego onto route so InRouteTest doesn't fail
    snap_ego_to_route_start(manager)
    world.tick(); world.tick()

    # Reset blackboard key if your tree uses it (optional but often needed)
    try:
        py_trees.blackboard.Blackboard().set("ScenarioRouteNumber0", None, overwrite=True)
    except Exception:
        pass

    # Reset the py_trees statuses
    reset_tree(manager.scenario_tree)

    if changed and not keep_sync:
        world.apply_settings(prev)

    start_manager_builder_only(manager)

# def pause_restore_resume(client, world, manager, vehicles_state, keep_sync=True):
#     """
#     client: carla.Client (e.g., leaderboard_evaluator.client)
#     world : carla.World  (e.g., leaderboard_evaluator.manager.scenario.world)
#     """

#     # 1) Pause any manager activity
#     stop_manager(manager)

#     # 2) Ensure the world is paused (sync mode)
#     prev = world.get_settings()
#     changed = False
#     if not prev.synchronous_mode:
#         new = carla.WorldSettings()
#         new.no_rendering_mode   = prev.no_rendering_mode
#         new.synchronous_mode    = True
#         new.fixed_delta_seconds = prev.fixed_delta_seconds or 0.05
#         world.apply_settings(new)
#         changed = True

#     # 3) Batch teleport + zero velocities (atomic)
#     batch = []
#     for rec in vehicles_state:
#         vid = int(rec["id"])
#         tf  = carla.Transform(
#             carla.Location(float(rec["x"]), float(rec["y"]), float(rec["z"])),
#             carla.Rotation(
#                 yaw=float(rec.get("yaw",   0.0)),
#                 pitch=float(rec.get("pitch", 0.0)),
#                 roll=float(rec.get("roll",  0.0))
#             )
#         )
#         batch += [
#             carla.command.SetSimulatePhysics(vid, False),
#             carla.command.ApplyTransform(vid, tf),
#             carla.command.ApplyTargetVelocity(vid, carla.Vector3D(0,0,0)),
#             carla.command.ApplyTargetAngularVelocity(vid, carla.Vector3D(0,0,0)),
#             carla.command.SetSimulatePhysics(vid, True),
#         ]

#     snap_ego_to_route_start(manager)

#     client.apply_batch_sync(batch, True)   # True == do_tick in 0.9.15
#     world.tick(); world.tick()             # settle sensors/frames

#     reset_tree(manager.scenario_tree)

#     # 4) Optionally revert to previous async mode
#     if changed and not keep_sync:
#         world.apply_settings(prev)

#     # 5) Resume ONLY the builder loop; you will tick manually
#     start_manager_builder_only(manager)


In [ ]:

# def record_vehicles(world_obj, save_path=None, save_json=False):

#     actors = world_obj.get_actors()
#     vehicles = actors.filter("vehicle.*")

#     records = []
#     for v in vehicles:
#         tf = v.get_transform()
#         loc, rot = tf.location, tf.rotation
#         record = {
#             "id": v.id,
#             "type_id": v.type_id,
#             "x": loc.x,
#             "y": loc.y,
#             "z": loc.z,
#             "pitch": rot.pitch,
#             "yaw": rot.yaw,
#             "roll": rot.roll
#         }
#         records.append(record)

#     if save_path:
#         if save_json:
#             with open(save_path, "w") as f:
#                 json.dump(records, f, indent=4)
#             print(f"Saved {len(records)} vehicle records to {save_path} (JSON)")
#         else:
#             with open(save_path, mode="w", newline="") as f:
#                 writer = csv.DictWriter(
#                     f, 
#                     fieldnames=["id", "type_id", "x", "y", "z", "pitch", "yaw", "roll"]
#                 )
#                 writer.writeheader()
#                 writer.writerows(records)
#             print(f"Saved {len(records)} vehicle records to {save_path} (CSV)")

#     return records

In [ ]:
# Let's find what variables are available that could contain velocity information
import json

# Read the notebook file
with open('/mnt3/Documents/AD_Framework/bench2drive-gymnasium/bench2drive_microservices/notebooks/step_by_step_leaderboard_snapshot_restore2.ipynb', 'r') as f:
    notebook = json.load(f)

# Find the cell with save_debug_images function
for i, cell in enumerate(notebook['cells']):
    if cell['cell_type'] == 'code':
        source = ''.join(cell['source'])
        if 'def save_debug_images' in source:
            print(f"Found save_debug_images in cell {i}")
            print(f"Cell has {len(cell['source'])} lines")
            # Check what's available in the function context
            for j, line in enumerate(cell['source']):
                if 'input_data' in line or 'GPS' in line or 'SPEED' in line:
                    print(f"  Line {j}: {line.strip()}")
            break

In [ ]:
# Let's check what sensor data is typically available
import json

with open('/mnt3/Documents/AD_Framework/bench2drive-gymnasium/bench2drive_microservices/notebooks/step_by_step_leaderboard_snapshot_restore2.ipynb', 'r') as f:
    notebook = json.load(f)

# Look for sensor types mentioned in the notebook
for i, cell in enumerate(notebook['cells']):
    if cell['cell_type'] == 'code':
        source = ''.join(cell['source'])
        if 'GPS' in source or 'IMU' in source or 'SPEED' in source:
            lines = [line for line in cell['source'] if any(sensor in line for sensor in ['GPS', 'IMU', 'SPEED'])]
            if lines:
                print(f"Cell {i} mentions sensors:")
                for line in lines[:5]:  # First 5 matching lines
                    print(f"  {line.strip()}")

In [ ]:
import json

with open('/mnt3/Documents/AD_Framework/bench2drive-gymnasium/bench2drive_microservices/notebooks/step_by_step_leaderboard_snapshot_restore2.ipynb', 'r') as f:
    notebook = json.load(f)

# Find the cell with save_debug_images function
for i, cell in enumerate(notebook['cells']):
    if cell['cell_type'] == 'code':
        source = ''.join(cell['source'])
        if 'def save_debug_images' in source:
            # Check if cell has an ID
            cell_id = cell.get('id', None)
            print(f"Found save_debug_images in cell index {i}")
            print(f"Cell ID: {cell_id}")
            print(f"First few lines:")
            for line in cell['source'][:5]:
                print(f"  {line.rstrip()}")
            break

In [ ]:

# def save_debug_images(agent_instance, save_dir, step_count):
#     """Save all sensor images for debugging"""
#     if not hasattr(agent_instance, 'last_input_data'):
#         return
    
#     input_data = agent_instance.last_input_data
    
#     # Create step-specific subdirectory
#     step_dir = save_dir / f"step_{step_count:04d}"
#     step_dir.mkdir(exist_ok=True)
    
#     # Save RGB cameras
#     for camera_id in ['Center', 'Left', 'Right']:
#         if camera_id in input_data:
#             rgb_image = input_data[camera_id][1][:, :, :3]  # Remove alpha channel if present
#             frame_num = input_data[camera_id][0]
            
#             # Convert from RGB to BGR for OpenCV
#             bgr_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR)
            
#             # Save image
#             image_path = step_dir / f"{camera_id}_frame_{frame_num:06d}.jpg"
#             cv2.imwrite(str(image_path), bgr_image)
#             # print(f"  Saved: {camera_id} -> {image_path.name}")
    
#     # Save sensor data info to text file
#     info_path = step_dir / "sensor_info.txt"
#     with open(info_path, 'w') as f:
#         f.write(f"Step: {step_count}\n")
#         f.write(f"Timestamp: {datetime.now()}\n\n")
#         f.write("Sensor Data:\n")
#         for key, val in input_data.items():
#             if hasattr(val[1], 'shape'):
#                 f.write(f"  {key}: frame={val[0]:06d}, shape={val[1].shape}\n")
#             else:
#                 f.write(f"  {key}: frame={val[0]:06d}\n")
    
#     return step_dir